In [ ]:
pip install fasttext

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Journée Entreprise (réponses) - Réponses au formulaire 1 (1).csv")
df2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Journée Entreprise (réponses) - Réponses au formulaire 1.csv")
df3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Competency Based Curriculum 4DS4 - Réponses au formulaire 1.csv")

In [ ]:
display("4DS6",df1
        ,"4DS8",df2
        ,"4DS4",df3)

In [ ]:
dataP=pd.concat([df1,df2,df3])
dataP

In [ ]:
dataP.drop('Score', axis=1, inplace=True)

In [ ]:
dataP

In [ ]:
dataP.shape

In [ ]:
dataP.to_csv("data2.csv",index=True)

## web scraping

In [ ]:
pip install cloudscraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import cloudscraper
import random


In [ ]:
def extract(page):
    headers={'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    #scraper = cloudscraper.create_scraper(delay=10,   browser={'custom': 'ScraperBot/1.0',})
    url=f'https://www.indeed.com/jobs?q=data+scientist&start={page}'
    r= requests.get(url,headers)
    return r.status_code
    #soup =BeautifulSoup(r.content,'html.parser')
    #return soup

In [ ]:
print(extract(20))

In [ ]:
c=extract(30)
c

In [ ]:
def transform(soup):
    divs=soup.find_all('div',class_='cardOutline tapItem ')
    return len(divs)

In [ ]:
print(transform(c))

### scraping with selenium

In [ ]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import time
from time import sleep
import pandas as pd
from random import randint

In [ ]:
website = """
#########################################
#           WEBSITE: INDEED.FR          #
######################################### 
"""
print(website)

In [ ]:
start_time = datetime.now()
print('Crawl starting time : {}' .format(start_time.time()))
print()
job_list = ["data+analyst", "data+scientist", "business+analyst"]

application_links = []
job_titles = []
company_names = []
job_locations = []
application_types = []
publication_dates = []
scraping_dates = []

In [ ]:
for job in job_list:

    opts = webdriver.FirefoxOptions()
    opts.headless = True
    profile = webdriver.FirefoxProfile('./firefoxdriver')
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)")
    driver = webdriver.Firefox(profile, executable_path="venv/bin/geckodriver", options=opts) 

    driver.get(
            "https://www.indeed.fr/jobs?q=" + job + "&l=Paris+%2875%29&jt=fulltime&limit=50&radius=25&start=0"
        )

    sleep(randint(7,10))
    print('Collecting data for "{}"...' .format(job))
    # First, get the number of jobs available
    job_number = driver.find_element_by_xpath("//div[@id='searchCountPages']").text
    # Calculating number of pages to be crawled (number of jobs available - number of jobs per page (here, 30))
    job_number = job_number.split(" ", 4)
    job_number = int(job_number[3])
    print("- Number of open positions : {}" .format(job_number))
    exact_page_nb = job_number / 50
    print("- Exact number of pages to be crawled : {}" .format(exact_page_nb))
    min_page_nb = job_number // 50
    print("- Minimum number of pages to be crawled : {}" .format(min_page_nb))

    if exact_page_nb > min_page_nb:
        page_nb = (min_page_nb) * 50
        pages = [str(i) for i in range(0, page_nb, 50)]
    elif exact_page_nb == min_page_nb:
        page_nb = (min_page_nb - 1) * 50
        pages = [str(i) for i in range(0, page_nb, 50)]

    for page in pages:
        driver.get(
            "https://www.indeed.fr/jobs?q=" + job + "&l=Paris+%2875%29&jt=fulltime&limit=50&radius=25&start=" + page
        )

        sleep(randint(5, 12))

        # Locating job container
        all_cards = driver.find_elements_by_xpath("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']")

        for card in all_cards:

            # Collecting job link
            application_link = card.find_elements_by_css_selector('a')
            if not application_link:
                application_link = "Unknown"
            else:
                application_link = application_link[0].get_attribute('href')
            application_links.append(application_link)

            # Collecting job title
            job_title = card.find_elements_by_css_selector('a')
            if not job_title:
                job_title = "Unknown"
            else:
                job_title = job_title[0].text
            job_titles.append(job_title)

            # Collecting company name
            company_name = card.find_elements_by_css_selector('div.sjcl div span.company')
            if not company_name:
                company_name = "Unknown"
            else:
                company_name = company_name[0].text
            company_names.append(company_name)
            
            # Collecting job location
            job_location = card.find_elements_by_css_selector('.location.accessible-contrast-color-location')
            if not job_location:
                job_location = "Unknown"
            else:
                job_location = job_location[0].text
            job_locations.append(job_location)
            
            # Collecting application type (easy apply)
            application_type = card.find_elements_by_css_selector('.jobCardShelfContainer')
            if not application_type:
                application_type = "company's website"
            else:
                application_type = application_type[0].text
            application_types.append(application_type) 
            
            # Collecting publication date
            publication_date = card.find_elements_by_css_selector('span.date')
            if not publication_date:
                publication_date = "il y a 40 minutes"
            else:
                publication_date = publication_date[0].text
            publication_dates.append(publication_date)
            
            # Collecting generated scraping time 
            scraping_dates.append(datetime.now())

    print('Crawling status for "{}" : Done' .format(job))
    print()

    driver.quit()

print('Crawling time : {}' .format(datetime.now() - start_time))
print('Dataframe successfuly created and exported')

In [ ]:
driver = webdriver.Firefox()

In [ ]:
pip install webdriver-manager

# Data Collection


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/BI_DATA.csv")
df2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/Cloud-Data.csv")
df3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/CloudTwinBi-Data.csv")
df4=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataScience&BI-Data.csv")
df5=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataScience-Data.csv")
df6=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/Iot.csv")
df7=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/NetworkSecurity-Data.csv")
df8=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING ESPRIT.csv")
df9=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING IOT.csv")
df10=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING WEB DEV.csv")
df11=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/result-3.csv")
df12=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/result-6.csv")
df13=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/SoftwareEngineer-Data.csv")
df14=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TouslesProfils-Data.csv")
df15=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TWIN-result-1.csv")

In [ ]:
display(df1,df2,df3)

In [ ]:
df2.shape

In [ ]:
df3.shape

In [ ]:
List=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]
dfP=pd.concat(List)

In [ ]:
dfP.shape

In [ ]:
dfP

In [ ]:
dfP.to_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TTDataScraping.xlsx",index=True)

## Data Activity

In [ ]:
dfActivity1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS1_Data.csv")
dfActivity2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS2_Data.csv")
#dfActivity3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS4_Data.csv")
dfActivity4=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS6_Data.csv")
dfActivity5=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS9_Data.csv")
dfActivity6=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/Data-4DS8.csv")
dfActivity7=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/dataset_InterviewAlumni.csv")


In [ ]:
ListActivity=[dfActivity1,dfActivity2,dfActivity4,dfActivity5,dfActivity6,dfActivity7]
dfPActivity=pd.concat(ListActivity)

In [ ]:
dfPActivity.shape

In [ ]:
dfPActivity.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/TTDataActivity.csv",index=True)

In [ ]:
dfPActivity.to_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/TTDataActivity.xlsx",index=True)

In [ ]:
dfPActivity.head()

In [ ]:
dfP

In [ ]:
dfP.columns

In [ ]:
dfPActivity

In [ ]:
dfPActivity.columns

### dfP :dataset de scraping 
### dfPActivity :dataset de formulaire contient les activity

In [ ]:
dfP.columns

In [ ]:
dfP["description"].head()

## Categorical Features


In [ ]:
cat_features = dfP.select_dtypes(include=['object','bool']).columns.tolist()
cat_features

## Numerical Features

In [ ]:
num_features = dfP.select_dtypes(exclude=['object','bool']).columns.tolist()
num_features

In [ ]:
dfP= dfP.drop(columns=num_features)


In [ ]:
dfP

### ==> Google Translate has limits on volume translated. EasyNMT is a scalable solution.

In [ ]:
pip install fasttext

In [ ]:
pip install -U  easynmt


In [ ]:
#dfP["headline"] = dfP.apply(lambda dfP: model.translate(dfP["headline"], target_lang="en"), axis=1)
#model.translate(list1,target_lang="en")

In [ ]:
cat_features

In [ ]:
dd=dfP["headline"].dropna()

In [ ]:
dfP

In [ ]:
dfP.shape

In [ ]:
dd=dfP.fillna("unknown")

In [ ]:
dd

In [ ]:
dd

In [ ]:
from easynmt import EasyNMT
model = EasyNMT("opus-mt")

In [ ]:
len(dd)

In [ ]:
df1

In [ ]:
num_features = df1.select_dtypes(exclude=['object','bool']).columns.tolist()
num_features

In [ ]:
df1 = df1.drop(columns=num_features)

# print the resulting DataFrame
print(df1)

In [ ]:
df11=df1.fillna("unknown")

In [ ]:
dd.columns

In [ ]:
##columns=[]
Df=dd[['jobTitle','skill1','skill2','skill3','skill4','skill5', 'skill6', 'jobDescription','headline','jobTitle2','jobDescription2','currentJob', 'pastJob','job']]

In [ ]:
Df.describe()

In [ ]:
from easynmt import EasyNMT
# Create the EasyNMT translator
translator = EasyNMT('opus-mt')

# Translate a sentence
for col in Df.columns:
    Df[col] = Df[col].apply(lambda x: model.translate(x, source_lang='fr', target_lang='en'))

# Print the resulting DataFrame
print(Df)


  

In [ ]:
df11

In [ ]:
##Df.to_excel('C:/Users/Yahyaoui/Desktop/taba3hedha/Data_eng.xlsx', index=False)
import csv

with open('my_dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    for row in Df:
        writer.writerow(row)

In [ ]:
##df1=Df.copy()
##df1 = df1.loc[~(df1 == "unckown").all(axis=1)]
df1.to_excel('datar.xlsx', index=False)

In [ ]:
df1.shape



```
# Ce texte est au format code
```

# Pre-Traitement


In [ ]:
##Data_pre = pd.read_excel('datar.xlsx')
##Data_pre

In [ ]:
import re

In [ ]:
Data_pre.columns

In [ ]:
for col in ['skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6']:
  Data_pre[col] = Data_pre[col].map(lambda x: re.sub('[,\.!?]()', '', x))
  

In [ ]:
skills=['skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6']

In [ ]:
html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
for col in skills:
  Data_pre[col] = Data_pre[col].map(lambda x: x.lower())
  Data_pre[col] = Data_pre[col].apply(lambda x: re.sub(html,'', x))
  Data_pre[col] = Data_pre[col].apply(lambda x: re.sub('\n','', x))
  Data_pre[col] = Data_pre[col].apply(lambda x: re.sub(r'[^\x00-\x7f]',r'', x))

  

In [ ]:
from wordcloud import WordCloud
def create_word_cloud(target_df, column_name):
  print('Joining all words into long text....')
  full_text = ','.join(list(target_df[column_name].values))
  wordcloud = WordCloud(background_color="black", 
                        max_words=100,  # top 100 words in the
                        contour_width=2, 
                        contour_color='yellow')
  print('Creating word cloud')
  wordcloud.generate(full_text)
  return wordcloud

In [ ]:
##for col in skills:

wordcloud = create_word_cloud(Data_pre,'skill2')
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['has', 'been', 're', 'com', 'edu', 'use'])

In [ ]:
# If you set deacc=True which will removes the punctuations (that we already removed)
def convert_sentences_to_words(sentences):
    for sentence in sentences:        
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
def remove_all_stop_words(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
text_to_list = Data_pre.skill1.values.tolist()

In [ ]:
text_as_words = list(convert_sentences_to_words(text_to_list))

In [ ]:
len(text_as_words)
print(text_as_words[0])

In [ ]:
clean_words = remove_all_stop_words(text_as_words)

In [ ]:
# Note if you will use higher threshold, which will return the fewer phrases.
bigram = gensim.models.Phrases(clean_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[clean_words], threshold=100)  

In [ ]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Importing spacy and Loading model
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

In [ ]:
# Form Bigrams
clean_words_bigrams = make_bigrams(clean_words)

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
clean_words_lemmatized = lemmatization(clean_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
print(clean_words_lemmatized[:1][0][:100])

In [ ]:
print(len(clean_words))
print(len(clean_words[0]))
print(len(clean_words[99]))

In [ ]:
print(len(clean_words_lemmatized))
print(len(clean_words_lemmatized[0]))
print(len(clean_words_lemmatized[99]))

In [ ]:
import gensim.corpora as corpora

In [ ]:
id2word = corpora.Dictionary(clean_words_lemmatized)

In [ ]:
# Creating Corpus for the clean words
texts = clean_words_lemmatized

In [ ]:
# Creating The Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[:1][0][:100])

In [ ]:
##corpus

In [ ]:
clean_words_lemmatized[0]

In [ ]:
##POS Tagging
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
#input liste de liste de tokens
def pos_tagging_nltk(tokens):
    return [nltk.pos_tag(w) for w in tokens if w]

In [ ]:
pos_tagging_nltk(clean_words_lemmatized)

## PreTraitement NLP Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dfP1=pd.read_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datar.xlsx")

In [ ]:
dfAC=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/dataactivityEn.csv")

In [ ]:
dfP1

In [ ]:
dfP1.drop(columns="Unnamed: 0",inplace=True)

In [ ]:
dfP1.columns

# Extracting Deck from Cabin
new_data['Cabin_Deck'] = data['Cabin'].str.split('/').str[0]

# Extracting Num from Cabin
new_data['Cabin_Num'] = data['Cabin'].str.split('/').str[1]

# Extracting Side from Cabin
new_data['Cabin_Side'] =data['Cabin'].str.split('/').str[2]

new_data.head()

In [ ]:
dfP1["jobDescription"].value_counts()

In [ ]:
dfP1["jobDescription2"].value_counts()

In [ ]:
dfPD=dfP1[['jobDescription','jobDescription2']]
dfPD

In [ ]:
dfPS=dfP1[['jobTitle', 'skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6']]
dfPS

### tajrib extraction des activity mn jobDescription a travers model pretrained 

In [ ]:
dfPD

In [ ]:
pip install spacy

In [ ]:
pip install --upgrade spacy

In [ ]:
python -m spacy download en_core_web_sm

In [ ]:
pip install spacy download en_core_web_sm

In [ ]:
import spacy

# Load the pre-trained model
nlp = spacy.load("en_core_web_sm")

# Define the job description paragraph
job_description = "The ideal candidate will be responsible for managing projects, coordinating with stakeholders, and ensuring timely delivery of results."

# Parse the job description with spaCy
doc = nlp(job_description)

# Extract activities from the text
activities = []
for ent in doc.ents:
    if ent.label_ == "ACTIVITY":
        activities.append(ent.text)

# Print the extracted activities
print(activities)

In [ ]:
pip install transformers


In [ ]:
pip install torch

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch

# Load the tokenizer and pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Define your dataset
texts = ['I went for a hike in the mountains.',
         'I went to the gym and did some weightlifting.',
         'I played tennis with my friends.',
         'I went swimming at the beach.',
         'I went for a run in the park.']
labels = [1, 2, 3, 4, 5]  # hiking, weightlifting, tennis, swimming, running

# Tokenize the input texts
encodings = tokenizer(texts, truncation=True, padding=True)

# Convert the labels to a tensor
labels = torch.tensor(labels)

# Define the optimizer and the training dataset
optimizer = AdamW(model.parameters(), lr=1e-5)
dataset = torch.utils.data.TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)

# Fine-tune the BERT model
model.train()
for epoch in range(3):
    for batch in torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True):
        optimizer.zero_grad()
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Test the model on some example texts
test_texts = ['I went for a swim in the pool.', 'I went to the park and played frisbee.']
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')
model.eval()
with torch.no_grad():
    inputs = {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']}
    outputs = model(**inputs)
    predictions = outputs.logits.softmax(dim=-1)
    predicted_labels = predictions.argmax(dim=-1)

# Print the predicted labels
for i, text in enumerate(test_texts):
    print(f'Text: {text}')
    print(f'Predicted activity: {predicted_labels[i].item() + 1}')  # add 1 to match label encoding


In [ ]:
from transformers import pipeline

# Load the NER model
ner_model = pipeline('ner', model='bert-base-cased', tokenizer='bert-base-cased')

# Define your input text
input_text = 'I went hiking in the mountains with my friends last weekend.'

# Extract entities from the input text
##ner_output = ner_model(df3['jobDescription'][1047])
ner_output = ner_model(input_text)

# Print the full NER output
print(ner_output)

# Extract the activity from the input text
activity = ''
for item in ner_output:
    if item['entity'] == 'ACTIVITY':
        activity += item['word'] + ' '

print(f'Activity: {activity.strip()}')


In [ ]:
!pip install python-rake==1.4.4

In [ ]:
import RAKE 
import operator

In [ ]:
sa=df3['jobDescription'][8]

In [ ]:
!pip install Rake-nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from rake_nltk import Rake
r=Rake()
r.extract_keywords_from_text(sa)
r.get_ranked_phrases()[0:5]
r.get_ranked_phrases_with_scores()[0:5]

In [ ]:
sa

In [ ]:
pip install pke


In [ ]:
import pke
from nltk.corpus import stopwords

In [ ]:
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None,
                        stoplist=stoplist)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
stoplist = stopwords.words('english')
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=3, threshold=threshold)

keyphrases
#[('python', 0.009552268191986027),
# ('interpreted high-level general-purpose', 0.03511690544258559),
# ('high-level general-purpose programming', 0.061741455019419696)]


### fin

### pretraitement de jobDescription

In [ ]:
dfPD.loc[1043].value_counts()

In [ ]:
len(dfPD["jobDescription"])

In [ ]:
new_data=dfPD.copy()

In [ ]:
#new_data['jobDescription'] = new_data['jobDescription'].str.split('\n').str[0]

In [ ]:
#new_data.iloc[1043:1044]

In [ ]:
#len(new_data['jobDescription'])

In [ ]:
# Extracting activity from jobdescription
new_data['Activity'] = new_data['jobDescription'].str.split('\n')

In [ ]:
new_data['Activity'].loc[1043]

In [ ]:
new_data['Activity']

In [ ]:
new_data

In [ ]:
new_data['Activity'].loc[1]

In [ ]:
new_data1=new_data.copy()

In [ ]:
# create new columns based on the length of the list
for i in range(10):
    col_name = f'Activite{i+1}'
    new_data1[col_name] = new_data1['Activity'].apply(lambda x: x[i] if len(x) > i else '')

# print the updated dataset
new_data1

In [ ]:
#same division of activity verticalement
#new_data1 = new_data1.explode('Activity')


In [ ]:
#new_data1['Activity'][1046]

In [ ]:
new_data1

In [ ]:
new_data1.drop("jobDescription2",axis=1,inplace=True)

In [ ]:
new_data1

### pretraitement de jobDescription2

In [ ]:
new_data2=dfPD.copy()

In [ ]:
# Extracting activity from jobdescription2
new_data2['Activity2'] = new_data2['jobDescription2'].str.split('\n')

In [ ]:
#new_data2 = new_data2.explode('Activity2')


In [ ]:
new_data2

In [ ]:
new_data2.drop(columns="jobDescription",inplace=True)

In [ ]:
new_data2

In [ ]:
print(type(new_data2["Activity2"][1043]))

In [ ]:
# create new columns based on the length of the list
for i in range(10):
    col_name2 = f'Activite{i+1}'
    new_data2[col_name2] = new_data2['Activity2'].apply(lambda x: x[i] if isinstance(x, str) and len(x) > i else '')

# print the updated dataset
new_data2

In [ ]:
new_data2.loc[1047]

In [ ]:
new_data2["Activity2"].loc[1043]

In [ ]:
new_data2 = new_data2.explode('Activity2')
new_data2

#new_data2.drop(columns=['jobDescription','jobDescription2'],inplace=True)

new_data3=new_data1.append(new_data2)
new_data3.loc[1047]

In [ ]:
data_match=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/data_to_match.csv")

In [ ]:
data_match

In [ ]:
data_match.to_excel("dat.xlsx")

## Matching

In [ ]:
dfPS

In [ ]:
skills=dfPS.copy()

In [ ]:
s3=list(skills['skill1'])+list(skills['skill2'])+list(skills['skill3'])+list(skills['skill4'])+list(skills['skill5'])+list(skills['skill6'])

In [ ]:
df = pd.DataFrame(s3)
skll=df[0].unique()
len(skll)

In [ ]:
df1=pd.DataFrame(skll)
df1

In [ ]:
pip install transformers


In [ ]:
pip install -U gensim

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models

# read the CSV file into a pandas dataframe
#data = pd.read_csv('text_data.csv')

# tokenize the text data using n-gram tokenization
ngram_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 5))
ngram_matrix = ngram_vectorizer.fit_transform(df1[0].tolist())
ngram_features = ngram_vectorizer.get_feature_names()

# load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# tokenize the text data and convert to input tensors
inputs = tokenizer(df1[0].tolist(), padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor([1, 0, 1, 0])  # sample labels for demonstration purposes

# fine-tune the BERT model on the text data
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(3):
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# get the predicted topic for each document
with torch.no_grad():
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    _, predicted = torch.max(outputs.logits, dim=1)

# create a dictionary and corpus for topic modeling using gensim
corpus = [tokenizer.tokenize(doc) for doc in df1[0].tolist()]

# create a bag of words model using the n-gram features
ngram_dict = corpora.Dictionary([doc.split() for doc in ngram_features])
ngram_corpus = [ngram_dict.doc2bow(doc.split()) for doc in ngram_features]

# train a Latent Dirichlet Allocation (LDA) model on the corpus
lda_model = models.ldamodel.LdaModel(corpus, num_topics=2, id2word=ngram_dict, passes=10)

# print the top topics and their associated keywords
for topic in lda_model.show_topics():
    print(topic)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
pip install rake_nltk

In [ ]:
from rake_nltk import Rake
r=Rake()
r.extract_keywords_from_text(df1)
r.get_ranked_phrases()[0:5]
r.get_ranked_phrases_with_scores()[0:5]

## concat of dataset

In [ ]:
dfAC

In [ ]:
dfAC.columns

In [ ]:
dfAC

dfPActivity["Activite1"]=dfPActivity["Pour chaque activité, merci de décrire 2 à 3 tâches: \nACTIVITE 1:"]
dfPActivity["Activite2"]=dfPActivity["ACTIVITE 2:"]
dfPActivity["Activite3"]=dfPActivity["ACTIVITE 3:"]
dfPActivity["Activite4"]=dfPActivity["ACTIVITE 4:"]
dfPActivity["Activite5"]=dfPActivity["ACTIVITE 5:"]


In [ ]:
dfAC.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
dfAC.columns

In [ ]:
dfAC.rename(columns={ 'Pour chaque activité, merci de décrire 2 à 3 tâches: \r\nACTIVITE 1:':'Activite1'}, inplace=True)
dfAC.rename(columns={ 'ACTIVITE 2:':'Activite2'}, inplace=True)
dfAC.rename(columns={ 'ACTIVITE 3:':'Activite3'}, inplace=True)
dfAC.rename(columns={ 'ACTIVITE 4:':'Activite4'}, inplace=True)
dfAC.rename(columns={ 'ACTIVITE 5:':'Activite5'}, inplace=True)




In [ ]:
dfAC.columns

In [ ]:
dfPActivity1=dfAC[['Poste actuel','Activite1',
       'Activite2', 'Activite3', 'Activite4', 'Activite5',]]
dfPActivity1

In [ ]:
new_df=pd.read_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datar.xlsx")

In [ ]:
new_df

In [ ]:
new_df.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
new_data1["Poste actuel	"]=new_df["jobTitle"]
new_data1.rename(columns={ 'Poste actuel\t':'Poste actuel'}, inplace=True)

#new_data1.drop("jobDescription",axis=1,inplace=True)
new_data1

In [ ]:
dataPrincipal=pd.concat([new_data1,dfPActivity1])
dataPrincipal

In [ ]:
dataPrincipal["Poste actuel"].value_counts()

In [ ]:
dataPrincipal.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/dataPrincipal1.csv")

## Pretraitement

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
datar=pd.read_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datar.xlsx")

In [ ]:
datar

In [ ]:
datar.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
datar

In [ ]:
dataPrincipal=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/dataPrincipal1.csv")

In [ ]:
dataPrincipal

In [ ]:
dataPrincipal.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
dataPrincipal

In [ ]:
def clean_text(raw):
    '''Case specific to be used with pandas apply method'''
    try:
        # remove carriage returns and new lines
        raw = raw.replace('\r', '')
        raw = raw.replace('\n', '')
        
        # brackets appear in all instances
        raw = raw.replace('[', '')
        raw = raw.replace(']', '')
        raw = raw.replace(')', '')
        raw = raw.replace('(', '')
        raw['clean_text'] = raw['clean_text'].apply(lambda x: re.sub(r'[^\x00-\x7f]',r'', x))
        
        # removing html tags
        clean_html = re.compile('<.*?>')
        clean_text = re.sub(clean_html, ' ', raw)
        
        # removing duplicate whitespace in between words
        clean_text = re.sub(" +", " ", clean_text) 
        
        # stripping first and last white space 
        clean_text = clean_text.strip()
        
        # commas had multiple spaces before and after in each instance
        clean_text = re.sub(" , ", ", ", clean_text) 
        
        # eliminating the extra comma after a period
        clean_text = clean_text.replace('.,', '.')
        
        # using try and except due to Nan in the column
    except:
        clean_text = np.nan
        
    return raw

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
dataPrincipal.columns

In [ ]:
dataPrincipal.columns

In [ ]:
dataPrincipal.fillna("unknown",inplace=True)

In [ ]:
dataPrincipal.replace('unckown','unknown',inplace=True)

In [ ]:
dataPrincipal

In [ ]:
clean_text(dataPrincipal)

In [ ]:
# regexp tokenizer eliminates the punctuation for the word count and is faster than word_tokenizer
tokenizer = RegexpTokenizer(r"\w+")
dataPrincipal['word_tokenized1'] = dataPrincipal['Activite1'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized2'] = dataPrincipal['Activite2'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized3'] = dataPrincipal['Activite3'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized4'] = dataPrincipal['Activite4'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized5'] = dataPrincipal['Activite5'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized6'] = dataPrincipal['Activite6'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized7'] = dataPrincipal['Activite7'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized8'] = dataPrincipal['Activite8'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized9'] = dataPrincipal['Activite9'].apply(lambda row: tokenizer.tokenize(row))
dataPrincipal['word_tokenized10'] = dataPrincipal['Activite10'].apply(lambda row: tokenizer.tokenize(row))




In [ ]:
dataPrincipal

In [ ]:
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

In [ ]:
new_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", "unknown"]

In [ ]:
def stopword_removal(text):
    #stop_words = set(stopwords.words('english'))
    cleaned = [word for word in text if word not in new_stopwords]
    return cleaned

In [ ]:
dataPrincipal['word_tokenized1'] = dataPrincipal.word_tokenized1.apply(stopword_removal)
dataPrincipal['word_tokenized2'] = dataPrincipal.word_tokenized2.apply(stopword_removal)
dataPrincipal['word_tokenized3'] = dataPrincipal.word_tokenized3.apply(stopword_removal)
dataPrincipal['word_tokenized4'] = dataPrincipal.word_tokenized4.apply(stopword_removal)
dataPrincipal['word_tokenized5'] = dataPrincipal.word_tokenized5.apply(stopword_removal)
dataPrincipal['word_tokenized6'] = dataPrincipal.word_tokenized6.apply(stopword_removal)
dataPrincipal['word_tokenized7'] = dataPrincipal.word_tokenized7.apply(stopword_removal)
dataPrincipal['word_tokenized8'] = dataPrincipal.word_tokenized8.apply(stopword_removal)
dataPrincipal['word_tokenized9'] = dataPrincipal.word_tokenized9.apply(stopword_removal)
dataPrincipal['word_tokenized10'] = dataPrincipal.word_tokenized10.apply(stopword_removal)

## Stemming and Lemmatization

In [ ]:
dataPrincipal['clean_words1'] = dataPrincipal.word_tokenized1.apply(stopword_removal)
dataPrincipal['clean_words2'] = dataPrincipal.word_tokenized2.apply(stopword_removal)
dataPrincipal['clean_words3'] = dataPrincipal.word_tokenized3.apply(stopword_removal)
dataPrincipal['clean_words4'] = dataPrincipal.word_tokenized4.apply(stopword_removal)
dataPrincipal['clean_words5'] = dataPrincipal.word_tokenized5.apply(stopword_removal)
dataPrincipal['clean_words6'] = dataPrincipal.word_tokenized6.apply(stopword_removal)
dataPrincipal['clean_words7'] = dataPrincipal.word_tokenized7.apply(stopword_removal)
dataPrincipal['clean_words8'] = dataPrincipal.word_tokenized8.apply(stopword_removal)
dataPrincipal['clean_words9'] = dataPrincipal.word_tokenized9.apply(stopword_removal)
dataPrincipal['clean_words10'] = dataPrincipal.word_tokenized10.apply(stopword_removal)

In [ ]:
def stemming(text):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in text]
    return stemmed

def lemming(text):
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(word) for word in text]
    return lemmed

In [ ]:
dataPrincipal['clean_stemmed1'] = dataPrincipal.clean_words1.apply(stemming)
dataPrincipal['clean_stemmed2'] = dataPrincipal.clean_words2.apply(stemming)
dataPrincipal['clean_stemmed3'] = dataPrincipal.clean_words3.apply(stemming)
dataPrincipal['clean_stemmed4'] = dataPrincipal.clean_words4.apply(stemming)
dataPrincipal['clean_stemmed5'] = dataPrincipal.clean_words5.apply(stemming)
dataPrincipal['clean_stemmed6'] = dataPrincipal.clean_words6.apply(stemming)
dataPrincipal['clean_stemmed7'] = dataPrincipal.clean_words7.apply(stemming)
dataPrincipal['clean_stemmed8'] = dataPrincipal.clean_words8.apply(stemming)
dataPrincipal['clean_stemmed9'] = dataPrincipal.clean_words9.apply(stemming)
dataPrincipal['clean_stemmed10'] = dataPrincipal.clean_words10.apply(stemming)

In [ ]:
nltk.download('omw-1.4')

In [ ]:
nltk.download('wordnet')

In [ ]:
dataPrincipal['clean_lemmed1'] = dataPrincipal.clean_words1.apply(lemming)
dataPrincipal['clean_lemmed2'] = dataPrincipal.clean_words2.apply(lemming)
dataPrincipal['clean_lemmed3'] = dataPrincipal.clean_words3.apply(lemming)
dataPrincipal['clean_lemmed4'] = dataPrincipal.clean_words4.apply(lemming)
dataPrincipal['clean_lemmed5'] = dataPrincipal.clean_words5.apply(lemming)
dataPrincipal['clean_lemmed6'] = dataPrincipal.clean_words6.apply(lemming)
dataPrincipal['clean_lemmed7'] = dataPrincipal.clean_words7.apply(lemming)
dataPrincipal['clean_lemmed8'] = dataPrincipal.clean_words8.apply(lemming)
dataPrincipal['clean_lemmed9'] = dataPrincipal.clean_words9.apply(lemming)
dataPrincipal['clean_lemmed10'] = dataPrincipal.clean_words10.apply(lemming)

In [ ]:
lem_corpus1 = []
for row in dataPrincipal.clean_lemmed1:
    lem_corpus1 += row
lem_corpus2 = []
for row in dataPrincipal.clean_lemmed2:
    lem_corpus2 += row
lem_corpus3 = []
for row in dataPrincipal.clean_lemmed3:
    lem_corpus3 += row
lem_corpus4 = []
for row in dataPrincipal.clean_lemmed4:
    lem_corpus4 += row
lem_corpus5 = []
for row in dataPrincipal.clean_lemmed5:
    lem_corpus5 += row
lem_corpus6 = []
for row in dataPrincipal.clean_lemmed6:
    lem_corpus6 += row
lem_corpus7 = []
for row in dataPrincipal.clean_lemmed7:
    lem_corpus7 += row
lem_corpus8 = []
for row in dataPrincipal.clean_lemmed8:
    lem_corpus8 += row

lem_corpus9 = []
for row in dataPrincipal.clean_lemmed9:
    lem_corpus9 += row
lem_corpus10 = []
for row in dataPrincipal.clean_lemmed10:
    lem_corpus10 += row

##  Fequent words

In [ ]:
pip install WordCloud

In [ ]:
lista=[lem_corpus1,lem_corpus2,lem_corpus3]

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud(
                          background_color='black',
                          max_words=100,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(lista))

fig = plt.figure(1, figsize = (20,15))
plt.imshow(wordcloud)
plt.axis('off')
wordcloud.to_file('lemma_wordcloud.png')

In [ ]:
def get_top_n_words(corpus):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq

In [ ]:
#Convert most freq words to dataframe for plotting bar plot
top_words = get_top_n_words(lem_corpus8)
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]

In [ ]:
#Barplot of most freq words
import seaborn as sns
lem_bar = sns.barplot(y = "Word", x = "Freq", data = top_df[:12], palette = "Blues_d", orient = 'h')
lem_bar.set_yticklabels(lem_bar.get_yticklabels(), fontsize = 19);
plt.title('Word Frequency In Entire Corpus', size=25)
plt.ylabel('Lemmatized Word', size = 25)
plt.xticks(size = 18)
plt.savefig('freqWords_bar.png', dpi=400)

### Tigrams

In [ ]:
#Most frequently occuring Tri-grams
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3), 
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

In [ ]:
# corpus with stopwords removed and list joined for n-gram distinction
ngram_corpus1 = dataPrincipal.clean_words1.str.join(' ')
ngram_corpus2 = dataPrincipal.clean_words2.str.join(' ')
ngram_corpus3 = dataPrincipal.clean_words3.str.join(' ')
ngram_corpus4 = dataPrincipal.clean_words4.str.join(' ')
ngram_corpus5 = dataPrincipal.clean_words5.str.join(' ')
ngram_corpus6 = dataPrincipal.clean_words6.str.join(' ')
ngram_corpus7 = dataPrincipal.clean_words7.str.join(' ')
ngram_corpus8 = dataPrincipal.clean_words8.str.join(' ')
ngram_corpus9 = dataPrincipal.clean_words9.str.join(' ')
ngram_corpus10 = dataPrincipal.clean_words10.str.join(' ')

In [ ]:
listaa=[ngram_corpus1,ngram_corpus2,ngram_corpus3,ngram_corpus4,ngram_corpus5,ngram_corpus6,ngram_corpus7,ngram_corpus8,ngram_corpus9,ngram_corpus10]

In [ ]:
top3_words = get_top_n3_words(ngram_corpus1, n = len(ngram_corpus1))
top3_df = pd.DataFrame(top3_words)
top3_df.columns = ["Tri-gram", "Freq"]

In [ ]:
#Barplot of most freq Tri-grams
plt.tight_layout()
sns.set(rc={'figure.figsize':(13,8)})
tbar = sns.barplot(y = "Tri-gram", x = "Freq", orient = 'h', data = top3_df[:14])
plt.title('Trigram Frequency In Entire Corpus', size=25)
plt.xticks(size = 18)
tbar.set_yticklabels(tbar.get_yticklabels(), fontsize = 20);
plt.savefig('trigram_bar.png', dpi=200)

In [ ]:
dataPrincipal.head(50)

In [ ]:
dataPrincipal.iloc[4]

In [ ]:
# get the indices of all observations equal to 'unknown'
for i in range(1112):
    #for j in range(12):
        unknown_indices = (dataPrincipal.iloc[i] == 'unknown' ).any().nonzero()
print(unknown_indices)
    
        



In [ ]:
dataPr=dataPrincipal.copy()

In [ ]:
dataPr.head(50)

In [ ]:
#dataPr.drop(dataPr.index[0],inplace=True)
#dataPr.drop(dataPr.index[3],inplace=True)
#dataPr.drop(dataPr.index[5],inplace=True)
#dataPr.drop(dataPr.index[6],inplace=True)
#dataPr.drop(dataPr.index[7],inplace=True)
#dataPr.drop(dataPr.index[9],inplace=True)
#dataPr.drop(dataPr.index[10],inplace=True)
#dataPr.drop(dataPr.index[11],inplace=True)
#dataPr.drop(dataPr.index[12],inplace=True)





In [ ]:
#listTotext=dataPr["Activity"]
#listTotext1=listTotext.tolist()
#listTotext1

### Text Generation

In [ ]:
listTotext1=["CRM","Dynamic 365","Power BI"]

In [ ]:
#All Libraries required 
import numpy
import sys
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras
import keras.utils
from keras import utils as np_utils
import numpy as np
import random
import sys
file = "Power BI"

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
words=word_tokenize(file)
words=" ".join(words)
words

In [ ]:
chars = sorted(list(set(words)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [ ]:
input_len = len(words)
vocab_len = len(chars)

In [ ]:
seq_length = 100
x_data = dataPr.iloc[4]
y_data = dataPr.iloc[4]

In [ ]:
for i in range(0, input_len - seq_length, 1):
    in_seq = words[i:i + seq_length]
    out_seq = words[i + seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns = len(x_data)

In [ ]:
y_data

In [ ]:
import pandas as pd
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)
y = np_utils.to_categorical(y_data)

In [ ]:
X=pd.DataFrame(x_data)
y=pd.DataFrame(y_data)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=4, batch_size=256)

### LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras
import keras.utils
from keras import utils as np_utils
import numpy as np
import random
import sys

In [ ]:
# Load the text data and preprocess it
txt="Power BI"
text = txt.lower()
#text = text.translate(str.maketrans('', '', text.punctuation))
chars = sorted(list(set(text)))
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))
max_sequence_length = 100
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - max_sequence_length, step):
    sentences.append(text[i:i + max_sequence_length])
    next_chars.append(text[i + max_sequence_length])
x = np.zeros((len(sentences), max_sequence_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(max_sequence_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
# Define a function to sample from the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Define a function to generate text using the model
def generate_text(seed, length=100, temperature=1.0):
    generated = seed
    for i in range(length):
        x_pred = np.zeros((1, max_sequence_length, len(chars)))
        for t, char in enumerate(seed):
            x_pred[0, t, char_to_index[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = index_to_char[next_index]
        generated += next_char
        seed = seed[1:] + next_char
    return generated

In [ ]:
# Define a callback function to generate text during training
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    seed = 'Power BI'
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        generated = generate_text(seed, length=100, temperature=temperature)
        print(generated)
        
    # Concatenate the generated text with the training history
    text_history.append(generated)
    text_history.append('\n')

# Initialize an empty list to store the generated text history
text_history = []

# Train the LSTM model and use the on_epoch_end function as a callback
model.fit(x, y, batch_size=128, epochs=20)

# Print the generated text history
print('Generated Text History:')
print(''.join(text_history))

### NLP: Text Generation through Bidirectional LSTM model

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [ ]:
tokenizer = Tokenizer()
#data = open('C:/Users/MSI/Desktop/Projet Data Science PI principal/test.txt').read()
data=" Data Science \n and Power BI."
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
print(input_sequences)

In [ ]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(predictors, label, epochs=100, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')
plt.figure()
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()
plt.show()

In [ ]:
dataPr

In [ ]:
seed_text = "Data Science and Power bi is the best."
next_words = 50

In [ ]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

### RNN for Text Generation

In [ ]:
pip install text_generator


In [ ]:
from text_generator import RNNTextGenerator
from model_selector import ModelSelector

# Fit the model
seq_length = 25
learning_rate = 0.01
epoch = 1
batch_size = 25
alice_dataset = "Power BI."
alice_model = RNNTextGenerator(
    seq_length,
    alice_dataset.vocab_size,
    rnn_cell=tf.nn.rnn_cell.BasicRNNCell,
    learning_rate=learning_rate,
    epoch=epoch,
    batch_size=batch_size,
)

alice_model_scores = alice_model.fit(
    alice_dataset,
    save_scores=True
)
fig, axes = plt.subplots(figsize=(15, 6), ncols=2)
alice_model_scores['accuracy'].plot(
    ax=axes[0], title='Accuracy (Train)'
)
alice_model_scores['loss'].plot(
    ax=axes[1], title='Loss (Train)'
)
for ax in axes:
    ax.set(xlabel='Steps')

# Traitement Principal

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
datar=pd.read_excel("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datar.xlsx")

In [3]:
datar

,Unnamed: 0,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription,headline,jobTitle2,jobDescription2,currentJob,pastJob,job
0,1,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unckown,ERP - Business Intelligence Engineering Studen...,Engineering student,unckown,unckown,unckown,unckown
1,2,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...,Business Intelligence Engineer,unckown,unckown,unckown,unckown,unckown
2,3,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro...",Business Intelligence Specialist Computer Engi...,Student,Skills: Engineering,unckown,unckown,unckown
3,4,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unckown,Business Intelligence Student,Trainee,unckown,unckown,unckown,unckown
4,5,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI,Business Intelligence engineering student at E...,Web developer,unckown,unckown,unckown,unckown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,139,VP Sales - Germany and Northern Europe,Strategy,Business Strategy,Leadership,Sales Strategy,Selling Solution,Go-to-market Strategy,In charge of selling Siradel solutions to tele...,I accompany telecom operators and smart cities...,OpenLegacy Sales Director France,I develop business with banks and insurance co...,unckown,unckown,unckown
1044,141,Technical Support Specialist,Budgeting,Administration,Team Management,Troubleshooting,Communication,Training,Skills : Troubleshooting · Communication · Cus...,IT Professional的 Increase Cross-Functional Tea...,Technical Support Specialist,Worked directly with L2 and engineering to add...,unckown,unckown,unckown
1045,142,Educational coordinator of the WEB competence ...,XML,E-business,Web Marketing,SEO,Business Strategy,Web 2.0,unckown,Teacher at ESPRIT (Ecole Supérieure Privée d'I...,Web teacher,unckown,unckown,unckown,unckown
1046,145,Software Engineer,Java,AngularJS,Laravel,SQL,PL/SQL,Scrum,Skills : Back-End Web Development · Agile Meth...,Computer engineer at VERMEG for Insurance & Fi...,Intern at VERMEG,unckown,unckown,unckown,unckown


In [4]:
datar.columns

Index(['Unnamed: 0', 'jobTitle', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'jobDescription', 'headline', 'jobTitle2',
       'jobDescription2', 'currentJob', 'pastJob', 'job'],
      dtype='object')

In [5]:
datar.drop("Unnamed: 0",axis=1,inplace=True)

In [6]:
datar

,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription,headline,jobTitle2,jobDescription2,currentJob,pastJob,job
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unckown,ERP - Business Intelligence Engineering Studen...,Engineering student,unckown,unckown,unckown,unckown
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...,Business Intelligence Engineer,unckown,unckown,unckown,unckown,unckown
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro...",Business Intelligence Specialist Computer Engi...,Student,Skills: Engineering,unckown,unckown,unckown
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unckown,Business Intelligence Student,Trainee,unckown,unckown,unckown,unckown
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI,Business Intelligence engineering student at E...,Web developer,unckown,unckown,unckown,unckown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,VP Sales - Germany and Northern Europe,Strategy,Business Strategy,Leadership,Sales Strategy,Selling Solution,Go-to-market Strategy,In charge of selling Siradel solutions to tele...,I accompany telecom operators and smart cities...,OpenLegacy Sales Director France,I develop business with banks and insurance co...,unckown,unckown,unckown
1044,Technical Support Specialist,Budgeting,Administration,Team Management,Troubleshooting,Communication,Training,Skills : Troubleshooting · Communication · Cus...,IT Professional的 Increase Cross-Functional Tea...,Technical Support Specialist,Worked directly with L2 and engineering to add...,unckown,unckown,unckown
1045,Educational coordinator of the WEB competence ...,XML,E-business,Web Marketing,SEO,Business Strategy,Web 2.0,unckown,Teacher at ESPRIT (Ecole Supérieure Privée d'I...,Web teacher,unckown,unckown,unckown,unckown
1046,Software Engineer,Java,AngularJS,Laravel,SQL,PL/SQL,Scrum,Skills : Back-End Web Development · Agile Meth...,Computer engineer at VERMEG for Insurance & Fi...,Intern at VERMEG,unckown,unckown,unckown,unckown


In [7]:
datar=datar[['jobTitle', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'jobDescription']]

In [8]:
datar.head()

,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unckown
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro..."
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unckown
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI


In [9]:
datar.replace('unckown','unknown',inplace=True)

C:\Users\MSI\anaconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [10]:
datar

,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unknown
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro..."
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unknown
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI
...,...,...,...,...,...,...,...,...
1043,VP Sales - Germany and Northern Europe,Strategy,Business Strategy,Leadership,Sales Strategy,Selling Solution,Go-to-market Strategy,In charge of selling Siradel solutions to tele...
1044,Technical Support Specialist,Budgeting,Administration,Team Management,Troubleshooting,Communication,Training,Skills : Troubleshooting · Communication · Cus...
1045,Educational coordinator of the WEB competence ...,XML,E-business,Web Marketing,SEO,Business Strategy,Web 2.0,unknown
1046,Software Engineer,Java,AngularJS,Laravel,SQL,PL/SQL,Scrum,Skills : Back-End Web Development · Agile Meth...


In [11]:
# Extracting activity from jobdescription
datar['Activity'] = datar['jobDescription'].str.split('\n')

<ipython-input-11-af9148f58d20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datar['Activity'] = datar['jobDescription'].str.split('\n')


In [12]:
datar.head()

,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription,Activity
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unknown,[unknown]
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...,[Design and implementation of dashboards to an...
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro...","[Skills: Extract, Transform, Load (ETL) · Micr..."
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unknown,[unknown]
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI,"[CRM, Dynamic 365, Power BI]"


In [13]:
# create new columns based on the length of the list
for i in range(10):
    col_name = f'Activite{i+1}'
    datar[col_name] = datar['Activity'].apply(lambda x: x[i] if len(x) > i else '')

# print the updated dataset
datar

<ipython-input-13-ebd6b9b6a1ca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datar[col_name] = datar['Activity'].apply(lambda x: x[i] if len(x) > i else '')


,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unknown,[unknown],unknown,,,,,,,,,
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...,[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unknown,[unknown],unknown,,,,,,,,,
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI,"[CRM, Dynamic 365, Power BI]",CRM,Dynamic 365,Power BI,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,VP Sales - Germany and Northern Europe,Strategy,Business Strategy,Leadership,Sales Strategy,Selling Solution,Go-to-market Strategy,In charge of selling Siradel solutions to tele...,[In charge of selling Siradel solutions to tel...,In charge of selling Siradel solutions to tele...,"3D representation, Digital twin, 5G, Smart cit...",,Siradel provides the Digital Twin of Territori...,,"Poland, Estonia, Germany, Latvia, Lithuania, F...",,,,
1044,Technical Support Specialist,Budgeting,Administration,Team Management,Troubleshooting,Communication,Training,Skills : Troubleshooting · Communication · Cus...,[Skills : Troubleshooting · Communication · Cu...,Skills : Troubleshooting · Communication · Cus...,,,,,,,,,
1045,Educational coordinator of the WEB competence ...,XML,E-business,Web Marketing,SEO,Business Strategy,Web 2.0,unknown,[unknown],unknown,,,,,,,,,
1046,Software Engineer,Java,AngularJS,Laravel,SQL,PL/SQL,Scrum,Skills : Back-End Web Development · Agile Meth...,[Skills : Back-End Web Development · Agile Met...,Skills : Back-End Web Development · Agile Meth...,,,,,,,,,


In [14]:
datar.columns

Index(['jobTitle', 'skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6',
       'jobDescription', 'Activity', 'Activite1', 'Activite2', 'Activite3',
       'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
       'Activite9', 'Activite10'],
      dtype='object')

In [15]:
# concatenate the six features into one feature
#datar['skills'] =list(datar.apply(lambda row: ''.join([str(row['skill1']), str(row['skill2']),
#                                                          str(row['skill3']), str(row['skill4']),
#                                                          str(row['skill5']), str(row['skill6'])]), axis=1))

In [16]:
datar['skills'] = datar.apply(lambda row: [row['skill1'], row['skill2'], row['skill3'], row['skill4'], row['skill5'], row['skill6']], axis=1)


In [17]:
datar.head()

,jobTitle,skill1,skill2,skill3,skill4,skill5,skill6,jobDescription,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10,skills
0,Business Intelligence Developer,SQL Server Integration Services (SSIS),SQL Server Management Studio,SQL,Spring Boot,Angular,Data analysis,unknown,[unknown],unknown,,,,,,,,,,"[SQL Server Integration Services (SSIS), SQL S..."
1,Internal Business Intelligence,Redmine,Microsoft Power BI,Talend Open Studio,R,Big data,Deep learning,Design and implementation of dashboards to ana...,[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,,"[Redmine, Microsoft Power BI, Talend Open Stud..."
2,Business Intelligence Trainee,"Extract, Transform, Load (ETL)",Leadership,Engineering,Project management,RStudio,.NET Core,"Skills: Extract, Transform, Load (ETL) · Micro...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,,"[Extract, Transform, Load (ETL), Leadership, E..."
3,Internship in engineering,Machine learning algorithms,PostgreSQL,Python (programming language),Project management,Java,.NET framework,unknown,[unknown],unknown,,,,,,,,,,"[Machine learning algorithms, PostgreSQL, Pyth..."
4,Development of a partner portal,E-commerce,Digital marketing,Decision-making informatics,Teamwork,Project management,Customer Relationship Management (CRM),CRM\nDynamic 365\nPower BI,"[CRM, Dynamic 365, Power BI]",CRM,Dynamic 365,Power BI,,,,,,,,"[E-commerce, Digital marketing, Decision-makin..."


In [18]:
datar.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datarD.csv")

In [19]:
datar.columns

Index(['jobTitle', 'skill1', 'skill2', 'skill3', 'skill4', 'skill5', 'skill6',
       'jobDescription', 'Activity', 'Activite1', 'Activite2', 'Activite3',
       'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
       'Activite9', 'Activite10', 'skills'],
      dtype='object')

In [20]:
datar1=datar[['jobTitle','skills' ,
        'Activity', 'Activite1', 'Activite2', 'Activite3',
       'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
       'Activite9', 'Activite10']]

In [21]:
datar1.head()

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,,,
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,
2,Business Intelligence Trainee,"[Extract, Transform, Load (ETL), Leadership, E...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
3,Internship in engineering,"[Machine learning algorithms, PostgreSQL, Pyth...",[unknown],unknown,,,,,,,,,
4,Development of a partner portal,"[E-commerce, Digital marketing, Decision-makin...","[CRM, Dynamic 365, Power BI]",CRM,Dynamic 365,Power BI,,,,,,,


In [ ]:
datar1.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/datarAcSki.csv")

In [22]:
#datar2 = datar1[datar1['jobTitle'] != 'unknown' or datar1['jobTitle'] != 'Student' or datar1['jobTitle'] != 'Student trainee' or datar1['jobTitle'] != 'Project trainee' or datar1['jobTitle'] != 'Trainee']

In [23]:
datar1['jobTitle'].value_counts()

unknown                                            98
Trainee                                            50
Student                                            39
Business Intelligence Internal                     28
Data Scientist                                     27
                                                   ..
Blockchain Internal                                 1
Data Science & Artificial Intelligence Internal     1
Blogger                                             1
Software consultant                                 1
In-company immersion course                         1
Name: jobTitle, Length: 496, dtype: int64

In [24]:
datar1 = datar1[datar1['jobTitle'] != 'unknown']
datar1=datar1[datar1['jobTitle'] != 'Student']
datar1=datar1[datar1['jobTitle'] != 'Trainee']
datar1=datar1[datar1['jobTitle'] != 'Internal']





In [25]:
datar1["jobTitle"].value_counts()

Business Intelligence Internal                     28
Data Scientist                                     27
Software Engineer                                  20
Business Intelligence Consultant                   16
Data Engineer                                      14
                                                   ..
Blockchain Internal                                 1
Data Science & Artificial Intelligence Internal     1
Blogger                                             1
Software consultant                                 1
In-company immersion course                         1
Name: jobTitle, Length: 492, dtype: int64

In [26]:
datar1.shape

(847, 13)

In [ ]:
datar1.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/data_2.csv")

In [ ]:
#datar2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/data_clusters2.csv")

In [ ]:
#datar2.head()

In [ ]:
#datar2.columns

In [ ]:
#datar2.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1,inplace=True)

In [27]:
datar1

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,,,
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,
2,Business Intelligence Trainee,"[Extract, Transform, Load (ETL), Leadership, E...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
3,Internship in engineering,"[Machine learning algorithms, PostgreSQL, Pyth...",[unknown],unknown,,,,,,,,,
4,Development of a partner portal,"[E-commerce, Digital marketing, Decision-makin...","[CRM, Dynamic 365, Power BI]",CRM,Dynamic 365,Power BI,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,VP Sales - Germany and Northern Europe,"[Strategy, Business Strategy, Leadership, Sale...",[In charge of selling Siradel solutions to tel...,In charge of selling Siradel solutions to tele...,"3D representation, Digital twin, 5G, Smart cit...",,Siradel provides the Digital Twin of Territori...,,"Poland, Estonia, Germany, Latvia, Lithuania, F...",,,,
1044,Technical Support Specialist,"[Budgeting, Administration, Team Management, T...",[Skills : Troubleshooting · Communication · Cu...,Skills : Troubleshooting · Communication · Cus...,,,,,,,,,
1045,Educational coordinator of the WEB competence ...,"[XML, E-business, Web Marketing, SEO, Business...",[unknown],unknown,,,,,,,,,
1046,Software Engineer,"[Java, AngularJS, Laravel, SQL, PL/SQL, Scrum]",[Skills : Back-End Web Development · Agile Met...,Skills : Back-End Web Development · Agile Meth...,,,,,,,,,


In [28]:
datar1.columns

Index(['jobTitle', 'skills', 'Activity', 'Activite1', 'Activite2', 'Activite3',
       'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
       'Activite9', 'Activite10'],
      dtype='object')

In [ ]:
#datar3=datar2[['jobTitle', 'skills', 'Activity', 'Activite1', 'Activite2', 'Activite3',
 #      'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
#       'Activite9', 'Activite10','cluster']]

In [ ]:
#datar3=datar2[['jobTitle', 'skills', 'Activity', 'Activite1', 'Activite2', 'Activite3',     'Activite4', 'Activite5', 'Activite6', 'Activite7', 'Activite8',
#       'Activite9', 'Activite10']]

In [ ]:
#datar3.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/data_clustersP.csv")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# Load dataset


# Extract description column
descriptions = datar1['jobTitle'].values

##descriptions = ngram_corpus10.values

# Convert descriptions to a TF-IDF matrix
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(descriptions)

# Calculate pairwise cosine similarity
cosine_similarities = cosine_similarity(X)

# Perform clustering using KMeans
n_clusters = 7  # specify the number of clusters
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(cosine_similarities)

# Add cluster labels to the DataFrame
datar1['cluster'] = kmeans.labels_

# Print the number of items in each cluster
print(datar1['cluster'].value_counts())

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans

# Load dataset


# Extract description column
descriptions = datar1['jobTitle'].values

##descriptions = ngram_corpus10.values

# Convert descriptions to a TF-IDF matrix
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(descriptions)

# Calculate pairwise Euclidean distance
distances = euclidean_distances(X)

# Perform clustering using KMeans
n_clusters = 6  # specify the number of clusters
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(distances)

# Add cluster labels to the DataFrame
datar1['cluster'] = kmeans.labels_

# Print the number of items in each cluster
print(datar1['cluster'].value_counts())

C:\Users\MSI\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


1    562
0    101
5     73
2     41
3     38
4     32
Name: cluster, dtype: int64


In [61]:
from nltk.corpus import wordnet

def get_wup_similarity(word1, word2):
    synset1 = wordnet.synsets(word1)
    synset2 = wordnet.synsets(word2)
    
    if synset1 and synset2:
        wup_similarity = synset1[0].wup_similarity(synset2[0])
        if wup_similarity is not None:
            return wup_similarity
    return 0

# Convert descriptions to a TF-IDF matrix using the WUP similarity measure
vectorizer = TfidfVectorizer(tokenizer=lambda text: text.split())
X = vectorizer.fit_transform(descriptions)

# Calculate pairwise Wu-Palmer Similarity measure
n = len(descriptions)
similarities = np.zeros((n, n))
for i in range(n):
    for j in range(i+1, n):
        doc1 = descriptions[i]
        doc2 = descriptions[j]
        wup_similarity = get_wup_similarity(doc1, doc2)
        similarities[i][j] = wup_similarity
        similarities[j][i] = wup_similarity

# Perform clustering using KMeans
n_clusters = 6  # specify the number of clusters
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(similarities)

# Add cluster labels to the DataFrame
datar1['cluster'] = kmeans.labels_

# Print the number of items in each cluster
print(datar1['cluster'].value_counts())


C:\Users\MSI\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\MSI\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


0    827
2     16
1      1
3      1
4      1
5      1
Name: cluster, dtype: int64


In [62]:
for i in range(n_clusters):
    print(f"Rows in cluster {i}:")
    cluster_rows = datar1[datar1['cluster'] == i]
    print(cluster_rows['jobTitle'])
    ##print()

Rows in cluster 0:
0                         Business Intelligence Developer
1                         Internal Business Intelligence 
2                           Business Intelligence Trainee
3                               Internship in engineering
4                         Development of a partner portal
                              ...                        
1043               VP Sales - Germany and Northern Europe
1044                         Technical Support Specialist
1045    Educational coordinator of the WEB competence ...
1046                                    Software Engineer
1047                                      Project Manager
Name: jobTitle, Length: 827, dtype: object
Rows in cluster 1:
887    Member
Name: jobTitle, dtype: object
Rows in cluster 2:
142        Teacher
190         Alumni
267        Blogger
433       Engineer
469      Treasurer
473         Member
565        Manager
613        Teacher
616      President
630      Treasurer
746        Manager
747      

In [32]:
for i in range(n_clusters):
    print(f"Rows in cluster {i}:")
    cluster_rows = datar1[datar1['cluster'] == i]
    print(cluster_rows['jobTitle'])
    ##print()

Rows in cluster 0:
0       Business Intelligence Developer
1       Internal Business Intelligence 
2         Business Intelligence Trainee
8       Business Intelligence Developer
12       Business Intelligence Internal
                     ...               
950     Business Intelligence Developer
954    Consultant business intelligence
957      Business Intelligence Internal
966               Business intelligence
968    Business Intelligence Consultant
Name: jobTitle, Length: 101, dtype: object
Rows in cluster 1:
3                               Internship in engineering
4                         Development of a partner portal
7                                            Data Analyst
9                                        Teacher trainer 
11                                          BI Consultant
                              ...                        
1042                 President - Managing Director France
1043               VP Sales - Germany and Northern Europe
1044           

In [33]:
datar1

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10,cluster
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,,,,0
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,,0
2,Business Intelligence Trainee,"[Extract, Transform, Load (ETL), Leadership, E...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,,0
3,Internship in engineering,"[Machine learning algorithms, PostgreSQL, Pyth...",[unknown],unknown,,,,,,,,,,1
4,Development of a partner portal,"[E-commerce, Digital marketing, Decision-makin...","[CRM, Dynamic 365, Power BI]",CRM,Dynamic 365,Power BI,,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,VP Sales - Germany and Northern Europe,"[Strategy, Business Strategy, Leadership, Sale...",[In charge of selling Siradel solutions to tel...,In charge of selling Siradel solutions to tele...,"3D representation, Digital twin, 5G, Smart cit...",,Siradel provides the Digital Twin of Territori...,,"Poland, Estonia, Germany, Latvia, Lithuania, F...",,,,,1
1044,Technical Support Specialist,"[Budgeting, Administration, Team Management, T...",[Skills : Troubleshooting · Communication · Cu...,Skills : Troubleshooting · Communication · Cus...,,,,,,,,,,1
1045,Educational coordinator of the WEB competence ...,"[XML, E-business, Web Marketing, SEO, Business...",[unknown],unknown,,,,,,,,,,1
1046,Software Engineer,"[Java, AngularJS, Laravel, SQL, PL/SQL, Scrum]",[Skills : Back-End Web Development · Agile Met...,Skills : Back-End Web Development · Agile Meth...,,,,,,,,,,5


In [34]:
datar1["cluster"].value_counts()

1    562
0    101
5     73
2     41
3     38
4     32
Name: cluster, dtype: int64

In [35]:
datar2=datar1.copy()

In [ ]:
# Extract description column
descriptions = cluster_data['jobTitle'].values


# Convert descriptions to a TF-IDF matrix
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(descriptions)

In [ ]:
# Calculate the similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(X)
similarity_matrix.shape

In [ ]:
cluster_data

In [ ]:
cluster_data.columns

In [ ]:
similarity_matrix.shape

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Compute the similarity matrix between jobTitle columns
vectorizer = CountVectorizer()
job_titles_matrix = vectorizer.fit_transform(cluster_data['jobTitle'])
similarity_matrix = cosine_similarity(job_titles_matrix)
print(similarity_matrix)
# Find the indices of rows that have a similarity of 1
indices = pd.np.where( (similarity_matrix >= 0.80) & (similarity_matrix<0.98))

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows = []

for i, j in zip(*indices):
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data.iloc[i], cluster_data.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
combined_df

#### similarity pearson

In [36]:
# Filter data to include only data points belonging to cluster 2
cluster_data = datar2[datar2['cluster'] == 0].drop(columns=['cluster'])
cluster_data

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,,,
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,,,
2,Business Intelligence Trainee,"[Extract, Transform, Load (ETL), Leadership, E...","[Skills: Extract, Transform, Load (ETL) · Micr...","Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
8,Business Intelligence Developer,"[Microsoft Power BI, Python (programming langu...",[Development of a BI solution to help detect f...,Development of a BI solution to help detect fr...,this solution based on 3 steps:,1/Data prepertion: data clearing with Python a...,2/Elaborate the modelling phase,3/Creating a predictive model,,,,,
12,Business Intelligence Internal,"[Telecommunications, Microsoft Word, Strategic...",[unknown],unknown,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,,,
954,Consultant business intelligence,"[Microsoft Power BI, SQL Server Integration Se...",[• Analyze and identify functional and technic...,• Analyze and identify functional and technica...,• Model and enrich the datawarehouse.,• Scheduling and deployment of jobs.,• Guidance: Training and mentoring of trainees...,,,,,,
957,Business Intelligence Internal,"[Microsoft Power BI, HTML5, Cascading Style Sh...",[I'm currently working on implementing a repor...,I'm currently working on implementing a report...,,,,,,,,,
966,Business intelligence,"[Microsoft Power BI, Project management, Manag...",[Skills: Microsoft Power BI · Project Management],Skills: Microsoft Power BI · Project Management,,,,,,,,,


In [37]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]

# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if (  (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data.iloc[i],cluster_data.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df22 = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
combined_df22

In [ ]:
combined_df22.jobTitle

In [38]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]

# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data.iloc[i],cluster_data.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df2 = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
combined_df2.jobTitle

In [ ]:
combined_df2

In [ ]:
combined_df2["Activity"]

In [39]:
combined_dfBI=pd.concat([combined_df2,combined_df22])

In [40]:
combined_dfBI

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,...,"Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,...,unknown,,,,,,,,,
2,Business Intelligence Developer,"[Microsoft Power BI, Python (programming langu...",[Development of a BI solution to help detect f...,Development of a BI solution to help detect fr...,this solution based on 3 steps:,1/Data prepertion: data clearing with Python a...,2/Elaborate the modelling phase,3/Creating a predictive model,,,...,unknown,,,,,,,,,
3,Business Intelligence Internal,"[DevOps, Angular, Spring Boot, Microsoft Power...",[Skills: Business Intelligence (BI) · Web Deve...,Skills: Business Intelligence (BI) · Web Devel...,,,,,,,...,Data modelling and data warehousing.,"""Implement ETL processes and pipelines.",Data Reporting and visualization.,的 Implementation supervised/unsupervised algor...,,,,,,
4,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,...,"Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Business Intelligence Consultant,"[ETL Tools, SQL, Business Intelligence (BI), S...",[Skills: ETL Tools · SQL · Business Intelligen...,Skills: ETL Tools · SQL · Business Intelligenc...,,,,,,,...,Consultant Business intelligence,"Qlik Sense, Qlik View, MSBI, Talend",,,,,,,,
18,Business Intelligence Internal,"[Microsoft Power BI, SQL Server Management Stu...",[Skills: Microsoft Power BI · SQL Server Manag...,Skills: Microsoft Power BI · SQL Server Manage...,,,,,,,...,Setting up a Datawarehouse:,• Preparation of technical and functional docu...,• Modelling and mapping data from different so...,• Development of SSIS packages to load databas...,• Planning of loading jobs.,• Optimization of SQL queries.,• Creating relationships with Power BI,Technology:,"- Microsoft BI (SSIS), SQL Server, Erwin, Powe...",
19,Business Intelligence Internal,"[Customer Relationship Management (CRM), Digit...",[Skills: Business Intelligence (BI) · Microsof...,Skills: Business Intelligence (BI) · Microsoft...,,,,,,,...,unknown,,,,,,,,,
20,Business Intelligence Trainee,"[ETL tools, Python (programming language), Mic...","[Project Internship End of Study: , -Realizati...",Project Internship End of Study:,-Realization of the functional study.,-Participation in all phases of ETL.,-Dashboard development.,-Prediction using Machine learning algorithms.,-Integrate the solution into a web interface u...,,...,• Analyze and identify functional and technica...,• Model and enrich the datawarehouse.,• Scheduling and deployment of jobs.,• Guidance: Training and mentoring of trainees...,,,,,,


In [ ]:
#cluster_cloud

In [41]:
# Filter data to include only data points belonging to cluster 2
cluster_data1 = datar2[datar2['cluster'] == 3].drop(columns=['cluster'])
cluster_data1

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
20,Cloud engineer and devops,"[Java, Cloud Computing, Git, Linux System Admi...",[unknown],unknown,,,,,,,,,
21,Cloud & DevOps Engineer,"[DevOps, Cloud Computing, Amazon Web Services ...",[unknown],unknown,,,,,,,,,
24,Senior Devops Engineer,[Continuous Integration and Continuous Deliver...,[unknown],unknown,,,,,,,,,
26,DevOps & Cloud Engineer,"[Go, Terraform, Ansible, Scrum, Docker, Angular]",[unknown],unknown,,,,,,,,,
28,DevOps Engineer,"[Tekton CI/CD, Gitlab, Continuous Integration ...",[unknown],unknown,,,,,,,,,
32,Cloud & DevOps Engineer,"[Cloud Computing, Cloud Security, OpenStack, B...",[Working with Microsoft Azure and its differen...,Working with Microsoft Azure and its different...,Migrating on-premises Applications to the clou...,Working With Azure Kubernetes Service both Man...,Writing DockerFiles and Generating Docker Images.,Developing CICD pipelines using Azure DevOps (...,Support of the Infrastructure and running Appl...,,,,
34,Internship Engineer DevOps,"[Cloud Computing, DevOps, AWS, Project managem...",[Continuous integration and deployment (CI/CD)...,Continuous integration and deployment (CI/CD) ...,,,,,,,,,
47,DevOps engineer,"[Network administration, Team spirit, Linux Sy...",[unknown],unknown,,,,,,,,,
53,Cloud & DevOps Engineer,"[Microsoft Office, Management, Microsoft Excel...",[Load test implementation for new features dev...,Load test implementation for new features deve...,Create Jenkins pipeline to automata test execu...,Analyze load test results to verify if there i...,Create Test Environments on AWS Using Terrafor...,Monitor production environment servers on AWS ...,Automatic several manual tasks using Jenkins p...,Automation and management test and production ...,,,
54,Cloud DevOps Engineer,"[Docker, Jenkins, Ansible, Cloud Computing, Ja...",[unknown],unknown,,,,,,,,,


In [42]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data1['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data1.iloc[i],cluster_data1.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df1 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.3440647518465709, -0.16226806099481, 0.014714436222104443, -0.16388670907903674, 0.3440647518465709, -0.15714451032761068, -0.14295576632501986, 0.3440647518465709, -0.01064881615828335, -0.16388670907903674, 0.014714436222104443, -0.07841435907695656, -0.14295576632501986, -0.16388670907903674, -0.16388670907903674, 0.3440647518465709, 0.3440647518465709, -0.16388670907903674, -0.13212104230987498, -0.16388670907903674, -0.2094017516173864, -0.16388670907903674, -0.16388670907903674, -0.11225138857089974, -0.16388670907903674, -0.13212104230987498, 0.3743403728592627, -0.07841435907695656, -0.14295576632501986, -0.16388670907903674, 1.0, -0.08378134932044462, -0.185985300764362, -0.16388670907903674, -0.16388670907903674, -0.08378134932044462, -0.13212104230987498], [0.3440647518465709, 1.0, 0.2541166784740195, 0.6875141592457696, -0.11632804800863825, 1.0, -0.19481630485914148, -0.18945178164314394, 1.0, 0.07412924001685028, -0.11632804800863825, 0.6875141592457696, 0.039728

In [43]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data1['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if (  (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data1.iloc[i],cluster_data1.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df11 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.3440647518465709, -0.16226806099481, 0.014714436222104443, -0.16388670907903674, 0.3440647518465709, -0.15714451032761068, -0.14295576632501986, 0.3440647518465709, -0.01064881615828335, -0.16388670907903674, 0.014714436222104443, -0.07841435907695656, -0.14295576632501986, -0.16388670907903674, -0.16388670907903674, 0.3440647518465709, 0.3440647518465709, -0.16388670907903674, -0.13212104230987498, -0.16388670907903674, -0.2094017516173864, -0.16388670907903674, -0.16388670907903674, -0.11225138857089974, -0.16388670907903674, -0.13212104230987498, 0.3743403728592627, -0.07841435907695656, -0.14295576632501986, -0.16388670907903674, 1.0, -0.08378134932044462, -0.185985300764362, -0.16388670907903674, -0.16388670907903674, -0.08378134932044462, -0.13212104230987498], [0.3440647518465709, 1.0, 0.2541166784740195, 0.6875141592457696, -0.11632804800863825, 1.0, -0.19481630485914148, -0.18945178164314394, 1.0, 0.07412924001685028, -0.11632804800863825, 0.6875141592457696, 0.039728

In [44]:
combined_dfCLOUD=pd.concat([combined_df1,combined_df11])

In [45]:
combined_dfCLOUD

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Cloud & DevOps Engineer,"[DevOps, Cloud Computing, Amazon Web Services ...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
1,Senior Devops Engineer,[Continuous Integration and Continuous Deliver...,[unknown],unknown,,,,,,,...,Skills: SQL · python · Git · Databases · Java ...,,,,,,,,,
2,DevOps Engineer,"[Tekton CI/CD, Gitlab, Continuous Integration ...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
3,Cloud & DevOps Engineer,"[Cloud Computing, Cloud Security, OpenStack, B...",[Working with Microsoft Azure and its differen...,Working with Microsoft Azure and its different...,Migrating on-premises Applications to the clou...,Working With Azure Kubernetes Service both Man...,Writing DockerFiles and Generating Docker Images.,Developing CICD pipelines using Azure DevOps (...,Support of the Infrastructure and running Appl...,,...,unknown,,,,,,,,,
4,Cloud & DevOps Engineer,"[Microsoft Office, Management, Microsoft Excel...",[Load test implementation for new features dev...,Load test implementation for new features deve...,Create Jenkins pipeline to automata test execu...,Analyze load test results to verify if there i...,Create Test Environments on AWS Using Terrafor...,Monitor production environment servers on AWS ...,Automatic several manual tasks using Jenkins p...,Automation and management test and production ...,...,Project:,- Talenteo is a native SIRH Cloud solution tha...,,Objective:,- Automation of CI/CD channels to deploy a nat...,,Tasks completed:,* Configure a pipeline to supply an Azure infr...,"* Three namespaces were set up on AKS (Dev, In...",* Deploy PostgreSQL in Statefulset on AKS usin...
5,DevOps Engineer,"[Node.js, React.js, MongoDB, Infrastructure, V...",[In this internship i worked on the infrastruc...,In this internship i worked on the infrastruct...,,,,,,,...,unknown,,,,,,,,,
6,DevOps Engineer,"[Infrastructure as code (IaC), Amazon Web Serv...",[Modernisation of Kubernetes resources - Best ...,Modernisation of Kubernetes resources - Best P...,Migration to a Highly Available and Secure GKE...,"Monitoring stack : Prometheus Operator, Promet...","Security stack: Certmanager, Letsencrypt...","Networking stack: Pomerium, Kubernetes Gateway...",,,...,Skills: Docker Products · Amazon Web Services ...,,,,,,,,,
7,DevOps Engineer,"[Cloud Development, Kubernetes, DevOps, Ansibl...",[unknown],unknown,,,,,,,...,Development and automation project,• Development an e-learning application.,• Containerize the application.,• Deploy application and DB to Azure.,• Automate CI/CD tasks with Jenkins and Azure ...,• Automatic configuration with ansible.,"• Tech :.Net, GitLab, Docke, Azure, Ansible, a...",,,
8,DevOps Engineer,"[Elastic Stack (ELK), Dynatrace, Prometheus.io...",[Build secure and automated high-performance c...,Build secure and automated high-performance co...,Establishment a CI/CD roadmap and implementing...,"Monitor, analytics, and report performance sta...",Experience with Software quality tools (Sonarq...,,,,...,Skills : Docker Products · Amazon Web Services...,,,,,,,,,
9,DevOps Engineer,"[Linux, C++, UML, C, Java, MySQL]",[• Ensure the steps of the CI/CD pipeline: Git...,• Ensure the steps of the CI/CD pipeline: Gith...,"• System support: Linux, Tomcat/Jboss, Apache,...","• Train, advise and assist users",• Ensure the preventive and curative maintenan...,"• Install, upgrade and upgrade hardware and so...",• Administer and maintain the network and its ...,• Ensuring the reliability and security of the...,...,unknown,,,,,,,,,


In [ ]:
#cluster_full_stack_developer

In [46]:
# Filter data to include only data points belonging to cluster 2
cluster_data2 = datar2[datar2['cluster'] == 4].drop(columns=['cluster'])
cluster_data2

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
115,Full Stack Developer,"[Git, PHP, HTML5, C#, Java, MySQL]",[Development of some features of CRM Client Re...,Development of some features of CRM Client Rel...,,,,,,,,,
123,Full-stack Web-Developer,"[HTML5, PHP, AngularJS, Java, Postman API, css]",[unknown],unknown,,,,,,,,,
256,Full stack data engineer,"[R, pandas, cloudera, SQL, MySQL, keras]",[unknown],unknown,,,,,,,,,
284,Full Stack Developer,"[Full-Stack Development, Data Science, SQL, Ma...",[unknown],unknown,,,,,,,,,
290,Senior Full Stack Web Developer,"[Java, MySQL, JavaScript, Angularjs, NodeJS, PHP]",[unknown],unknown,,,,,,,,,
299,Full Stack Consultant,"[Java, Python, Big Data, Agile methods, Softwa...",[unknown],unknown,,,,,,,,,
428,Full Stack Engineer,"[Redash, PostgreSQL, Django, Angular, redash, ...",[Skills: Django · Angular · redash · ClickHouse],Skills: Django · Angular · redash · ClickHouse,,,,,,,,,
451,Full-stack Developer,"[SQL Server Reporting Services (SSRS), SQL Ser...",[Skills: TypeScript · React.js],Skills: TypeScript · React.js,,,,,,,,,
497,Full stack and AI developer,"[JavaScript, MySQL, Deep Learning, Machine Lea...",[Skills: JavaScript · MySQL · Deep Learning · ...,Skills: JavaScript · MySQL · Deep Learning · M...,,,,,,,,,
550,Full Stack Developer,"[.NET framework, React.js, Node.js, Unity, Sec...",[Skills:.NET framework · AngularJS],Skills:.NET framework · AngularJS,,,,,,,,,


In [47]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data2['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data2.iloc[i],cluster_data2.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df2 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.30018428849852474, 0.34881113265904923, 1.0, -0.1270445859738878, 0.7668185242995278, 0.752658453015882, 0.9556548873139468, 0.004130963389529795, 1.0, 0.752658453015882, 1.0, 0.9556548873139468, 0.9556548873139468, 1.0, 1.0, 0.752658453015882, 0.9556548873139468, 0.9556548873139468, 0.9556548873139468, 1.0, 0.752658453015882, 0.752658453015882, 0.9556548873139468, 0.3197475920728018, 0.6632749228293839, 0.6632749228293839, 1.0, 0.9556548873139468, 1.0, 1.0, 1.0], [0.30018428849852474, 1.0, 0.8386902490799093, 0.30018428849852474, -0.3979728397642815, 0.3322324344873551, 0.21384940220129886, 0.30188623009086624, 0.42434130663669245, 0.30018428849852474, 0.21384940220129886, 0.30018428849852474, 0.30188623009086624, 0.30188623009086624, 0.30018428849852474, 0.30018428849852474, 0.21384940220129886, 0.30188623009086624, 0.30188623009086624, 0.30188623009086624, 0.30018428849852474, 0.21384940220129886, 0.21384940220129886, 0.30188623009086624, 0.6034861131215763, 0.2285460911676

In [48]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data2['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if (  (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data2.iloc[i],cluster_data2.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df22 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.30018428849852474, 0.34881113265904923, 1.0, -0.1270445859738878, 0.7668185242995278, 0.752658453015882, 0.9556548873139468, 0.004130963389529795, 1.0, 0.752658453015882, 1.0, 0.9556548873139468, 0.9556548873139468, 1.0, 1.0, 0.752658453015882, 0.9556548873139468, 0.9556548873139468, 0.9556548873139468, 1.0, 0.752658453015882, 0.752658453015882, 0.9556548873139468, 0.3197475920728018, 0.6632749228293839, 0.6632749228293839, 1.0, 0.9556548873139468, 1.0, 1.0, 1.0], [0.30018428849852474, 1.0, 0.8386902490799093, 0.30018428849852474, -0.3979728397642815, 0.3322324344873551, 0.21384940220129886, 0.30188623009086624, 0.42434130663669245, 0.30018428849852474, 0.21384940220129886, 0.30018428849852474, 0.30188623009086624, 0.30188623009086624, 0.30018428849852474, 0.30018428849852474, 0.21384940220129886, 0.30188623009086624, 0.30188623009086624, 0.30188623009086624, 0.30018428849852474, 0.21384940220129886, 0.21384940220129886, 0.30188623009086624, 0.6034861131215763, 0.2285460911676

In [49]:
combined_dfTWIN=pd.concat([combined_df2,combined_df22])

In [50]:
combined_dfTWIN

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Full Stack Developer,"[Git, PHP, HTML5, C#, Java, MySQL]",[Development of some features of CRM Client Re...,Development of some features of CRM Client Rel...,,,,,,,...,unknown,,,,,,,,,
1,Full-stack Web-Developer,"[HTML5, PHP, AngularJS, Java, Postman API, css]",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
2,Full Stack Developer,"[Full-Stack Development, Data Science, SQL, Ma...",[unknown],unknown,,,,,,,...,Skills: Django · Angular · redash · ClickHouse,,,,,,,,,
3,Full-stack Developer,"[SQL Server Reporting Services (SSRS), SQL Ser...",[Skills: TypeScript · React.js],Skills: TypeScript · React.js,,,,,,,...,Skills:.NET framework · AngularJS,,,,,,,,,
4,Full Stack Engineer,"[Scrum, Java, Linux System Administration, CSS...",[unknown],unknown,,,,,,,...,Realization of websites / web applications / M...,,,,,,,,,
5,Full-stack Developer,"[Laravel, vuejs, Angular, Node.js, Python (pro...",[development of an appointment website for hom...,development of an appointment website for home...,,,,,,,...,Competencies : Java · Angular,,,,,,,,,
6,Full-stack Developer,"[C (Programming Language), web3.js, Full-Stack...",[Skills : Angular · JavaScript],Skills : Angular · JavaScript,,,,,,,...,Design and realization of a web app,Laravel Framework,,,,,,,,
7,Full Stack Engineer,"[Scrum, Java, Linux System Administration, CSS...",[unknown],unknown,,,,,,,...,Project: Development and realization of a plat...,Participation in the various Scrum ceremonies ...,Design of the application database,Development of Back-end and Front-end games wi...,Generation of tickets on Jira,Unvelop APIs and test with Swagger,Management of anomalies and conflicts via Git,Fixed bugs and revised code to improve applica...,"Keywords :Docker, GitLab, PostgreSQL, Angular,...",
8,Full-stack Developer,"[AngularJS, React.js, Bootstrap, Node.js, Expr...",[unknown],unknown,,,,,,,...,• Led daily scrums to discussions current day-...,any impediments or concerns in an Agile Scrum ...,• Involved in all the internships of Software ...,"Design, Development, Debugging and Testing all...",• Build a Cross-Platform Mobile Application.,,,,,
9,Full-stack Developer,"[Quarkus, React.js, Symfony Framework, HTML5, ...",[unknown],unknown,,,,,,,...,Skills: Django · Angular · redash · ClickHouse,,,,,,,,,


In [ ]:
#cluster_dataAnalyst

In [ ]:
# Filter data to include only data points belonging to cluster 2
cluster_data3 = datar2[datar2['cluster'] == 4].drop(columns=['cluster'])
cluster_data3

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data3['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data3.iloc[i],cluster_data3.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df3 = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data3['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data3.iloc[i],cluster_data3.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df33 = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
combined_dfDATAANALYST=pd.concat([combined_df3,combined_df33])

In [ ]:
combined_dfDATAANALYST

In [ ]:
#cluster_dataScience

In [51]:
# Filter data to include only data points belonging to cluster 2
cluster_data4 = datar2[datar2['cluster'] == 2].drop(columns=['cluster'])
cluster_data4

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
176,Data Scientist Internal,"[Optical Character Recognition (OCR), Convolut...","[OCR Web service that detects, classifies the ...","OCR Web service that detects, classifies the o...",,,,,,,,,
180,Data Scientist,"[Python, Java, C++, C, MySQL, Unity]",[unknown],unknown,,,,,,,,,
185,Junior Data Scientist,"[c++, c/, java, Big Data, Algorithms, Computer...",[unknown],unknown,,,,,,,,,
188,Data Scientist,"[Business intelligence, R, Python, Time Series...",[unknown],unknown,,,,,,,,,
189,Data Scientist,"[Linux, Jira, Data Science, Python, machine le...",[• Customer Segmentation for a Retail Company:...,• Customer Segmentation for a Retail Company: ...,• Analyzed datasets and communicated insights ...,• Developed dashboards and reports that commun...,• Assessed business processes and recommended ...,• Developed a similar system to identify new e...,as well as classifying them according to fixin...,"Keywords : Python, Pandas, Numpy, Matplotlib, ...",,,
194,Data Scientist,"[Python, machine learning, Deep learning, Java...",[Development an application for lead generatio...,Development an application for lead generation.,• Web scraping to retrieve data from Job Searc...,Welcome to the jungle.,• Cleaning and normalization of recovered data...,"• Topic modelling (LDA, Bertopic).",• Creation of an application for the visualiza...,Generation.,"–Technologies : Python, Selenium, Beautiful So...","AI Algorithms, MongoDB, Power BI, Python Flask.",
195,Data scientist,"[Apache Airflow, Amazon Web Services, Machine ...",[Extract data from multiple sources and store ...,Extract data from multiple sources and store i...,,"Technologies : Python, SQL, MongoDB, AWS, AWS ...",,,,,,,
197,Data Engineer - Data Scientist,"[Databases, Machine Learning, Big Data Analysi...",[unknown],unknown,,,,,,,,,
198,Data Scientist,"[Apache Airflow, Microsoft Azure Machine Learn...",[-Extracting data from multiple sources and st...,-Extracting data from multiple sources and sto...,"Skills: S3, Glue, Athena, Redshift, Mixpanel, ...",-Executing ETL pipelines on wide data using Ap...,"-Measuring user engagement and resonant, worki...","Skills: Mixpanel, MongoCharts, Quicksight and ...",-Working close with customers and product team...,"-Training models for outfit recommendations, o...","Skills: pytorch, python, tensorflow, flask, fa...",,
199,Data Scientist,"[Google Cloud Platform (GCP), machine learning...",[unknown],unknown,,,,,,,,,


In [52]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data4['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data4.iloc[i],cluster_data4.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df4 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.3802944009703205, -0.08389674694295038, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3486207609578228, -0.0026033180947263868, 0.3802944009703205, 0.3802944009703205, 0.6602683067213575, 0.3802944009703205, -0.08389674694295038, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.12058569414772566, -0.030979100561054573, 0.3802944009703205, 0.05303896726100246, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, -0.10220680681920839, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, -0.05980808882542477, 0.3802944009703205, 0.3802944009703205, -0.10220680681920839, -0.10220680681920839, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205], [0.3802944009703205, 1.0, -0.3864654867740796, 1.0, 1.0, 1.0, 0.9321696536564104, 0.17257217830507632, 1.0, 1.0, 0.36056008825599206, 1.0, -0.3864654867740796, 1.0, 1.0, 1

In [53]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data4['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if (  (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data4.iloc[i],cluster_data4.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df44 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.3802944009703205, -0.08389674694295038, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3486207609578228, -0.0026033180947263868, 0.3802944009703205, 0.3802944009703205, 0.6602683067213575, 0.3802944009703205, -0.08389674694295038, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.12058569414772566, -0.030979100561054573, 0.3802944009703205, 0.05303896726100246, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, -0.10220680681920839, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, -0.05980808882542477, 0.3802944009703205, 0.3802944009703205, -0.10220680681920839, -0.10220680681920839, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205, 0.3802944009703205], [0.3802944009703205, 1.0, -0.3864654867740796, 1.0, 1.0, 1.0, 0.9321696536564104, 0.17257217830507632, 1.0, 1.0, 0.36056008825599206, 1.0, -0.3864654867740796, 1.0, 1.0, 1

In [54]:
combined_dfDATASCIENCE=pd.concat([combined_df4,combined_df44])
combined_dfDATASCIENCE

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Data Scientist Internal,"[Optical Character Recognition (OCR), Convolut...","[OCR Web service that detects, classifies the ...","OCR Web service that detects, classifies the o...",,,,,,,...,unknown,,,,,,,,,
1,Data Scientist,"[Python, Java, C++, C, MySQL, Unity]",[unknown],unknown,,,,,,,...,Extract data from multiple sources and store i...,,"Technologies : Python, SQL, MongoDB, AWS, AWS ...",,,,,,,
2,Junior Data Scientist,"[c++, c/, java, Big Data, Algorithms, Computer...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
3,Junior Data Scientist,"[Python, R, Machine Learning, Research, C#, Java]",[-Sentiment analysis on videos for a human res...,-Sentiment analysis on videos for a human reso...,-Skills suggestion system using Word2Vec (gensim),,,,,,...,unknown,,,,,,,,,
0,Data Scientist Internal,"[Optical Character Recognition (OCR), Convolut...","[OCR Web service that detects, classifies the ...","OCR Web service that detects, classifies the o...",,,,,,,...,unknown,,,,,,,,,
1,Junior Data Scientist,"[c++, c/, java, Big Data, Algorithms, Computer...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
2,Data Scientist,"[Linux, Jira, Data Science, Python, machine le...",[• Customer Segmentation for a Retail Company:...,• Customer Segmentation for a Retail Company: ...,• Analyzed datasets and communicated insights ...,• Developed dashboards and reports that commun...,• Assessed business processes and recommended ...,• Developed a similar system to identify new e...,as well as classifying them according to fixin...,"Keywords : Python, Pandas, Numpy, Matplotlib, ...",...,unknown,,,,,,,,,
3,Data Scientist,"[Python, machine learning, Deep learning, Java...",[Development an application for lead generatio...,Development an application for lead generation.,• Web scraping to retrieve data from Job Searc...,Welcome to the jungle.,• Cleaning and normalization of recovered data...,"• Topic modelling (LDA, Bertopic).",• Creation of an application for the visualiza...,Generation.,...,unknown,,,,,,,,,
4,Data scientist,"[Apache Airflow, Amazon Web Services, Machine ...",[Extract data from multiple sources and store ...,Extract data from multiple sources and store i...,,"Technologies : Python, SQL, MongoDB, AWS, AWS ...",,,,,...,-Sentiment analysis on videos for a human reso...,-Skills suggestion system using Word2Vec (gensim),,,,,,,,
5,Data Scientist,"[Apache Airflow, Microsoft Azure Machine Learn...",[-Extracting data from multiple sources and st...,-Extracting data from multiple sources and sto...,"Skills: S3, Glue, Athena, Redshift, Mixpanel, ...",-Executing ETL pipelines on wide data using Ap...,"-Measuring user engagement and resonant, worki...","Skills: Mixpanel, MongoCharts, Quicksight and ...",-Working close with customers and product team...,"-Training models for outfit recommendations, o...",...,Use of object detection models for extracting ...,- Annotation of the data.,- Increased data with GANs.,- Making an object detection model.,- Use of OCR for text extraction.,- Deployment of models in the form of a Backen...,,,,


In [ ]:
#cluster_GL

In [55]:
# Filter data to include only data points belonging to cluster 2
cluster_data5 = datar2[datar2['cluster'] == 5].drop(columns=['cluster'])
cluster_data5

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
19,Cloud Engineer,"[Cloud Computing, Software Development, Astrop...",[unknown],unknown,,,,,,,,,
31,Cloud Engineer,"[C++, Java, SQL, Linux, PHP, JavaScript]",[unknown],unknown,,,,,,,,,
38,Cloud Engineer,"[Big data, Business Intelligence, Amazon Web S...",[unknown],unknown,,,,,,,,,
50,Junior Software Engineer,"[DevOps, Cloud computing, Linux System Adminis...",[Developing an e-learning platform using sprin...,Developing an e-learning platform using spring...,,,,,,,,,
64,Cloud engineer,"[google G suite, Symfony Framework, GitHub, HT...",[*Contributes to the updating of a private clo...,*Contributes to the updating of a private clou...,*Operation of the various cloud infrastructure...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,Junior Software Engineer,"[MongoDB, JavaScript, PHP, Node.js, SQL No, Sy...",[unknown],unknown,,,,,,,,,
1001,Software Engineer,"[React, View.js, Java, React.js, Angular, Spri...",[Competencies : Java · React.js],Competencies : Java · React.js,,,,,,,,,
1021,Software Engineer,"[Algorithms, Apollo GraphQL, JavaScript, Node....",[achieve with the team the creating of NFT mar...,achieve with the team the creating of NFT mark...,"Keywords : hederaAPI, node js,express, prisma,...",,,,,,,,
1039,Software Engineer,"[React.js, Node.js, JavaScript, PHP, Java, Ang...",[unknown],unknown,,,,,,,,,


In [56]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data5['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data5.iloc[i],cluster_data5.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df5 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.9999999999999998, 1.0, 0.23395773558906388, 0.9334887315946319, 1.0, 1.0, -0.05815904324070921, 1.0, 1.0, -0.05815904324070921, 0.9334887315946319, 0.9334887315946319, 1.0, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, -0.05815904324070921, -0.05815904324070921, 0.18044735277969393, -0.05815904324070921, 0.18044735277969393, 0.18044735277969393, -0.15361911353761679, 0.18044735277969393, 0.18044735277969393, -0.05815904324070921, 0.18044735277969393, -0.21834034009177666, 0.1111020023157715, 0.0427313174614991, -0.05815904324070921, -0.3371646278928776, -0.05815904324070921, -0.030486924758962388, -0.14926471357396348, 0.15353388700628692, 0.22120555279194276, -0.21834034009177666, -0.10245196361216671, 1.0, -0.05815904324070921, 0.027715147517061722, -0.05815904324070921, 0.18044735277969393, 0.027715147517061722, -0.05815904324070921, -0.05815904324070921, -0.14926471357396348, -0.030486924758962

In [57]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data5['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if (  (similarity_matrix[i][j] < 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data5.iloc[i],cluster_data5.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df55 = pd.concat(unique_combined_rows, axis=1).transpose()


[[1.0, 0.9999999999999998, 1.0, 0.23395773558906388, 0.9334887315946319, 1.0, 1.0, -0.05815904324070921, 1.0, 1.0, -0.05815904324070921, 0.9334887315946319, 0.9334887315946319, 1.0, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, 0.18044735277969393, -0.05815904324070921, -0.05815904324070921, 0.18044735277969393, -0.05815904324070921, 0.18044735277969393, 0.18044735277969393, -0.15361911353761679, 0.18044735277969393, 0.18044735277969393, -0.05815904324070921, 0.18044735277969393, -0.21834034009177666, 0.1111020023157715, 0.0427313174614991, -0.05815904324070921, -0.3371646278928776, -0.05815904324070921, -0.030486924758962388, -0.14926471357396348, 0.15353388700628692, 0.22120555279194276, -0.21834034009177666, -0.10245196361216671, 1.0, -0.05815904324070921, 0.027715147517061722, -0.05815904324070921, 0.18044735277969393, 0.027715147517061722, -0.05815904324070921, -0.05815904324070921, -0.14926471357396348, -0.030486924758962

In [58]:
combined_dfGL=pd.concat([combined_df5,combined_df55])
combined_dfGL

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Cloud Engineer,"[Cloud Computing, Software Development, Astrop...",[unknown],unknown,,,,,,,...,*Contributes to the updating of a private clou...,*Operation of the various cloud infrastructure...,,,,,,,,
1,Cloud Engineer,"[C++, Java, SQL, Linux, PHP, JavaScript]",[unknown],unknown,,,,,,,...,Skills : English · DevOps · Microsoft Azure · ...,,,,,,,,,
2,Cloud Engineer,"[Big data, Business Intelligence, Amazon Web S...",[unknown],unknown,,,,,,,...,*Contributes to the updating of a private clou...,*Operation of the various cloud infrastructure...,,,,,,,,
3,Junior Software Engineer,"[DevOps, Cloud computing, Linux System Adminis...",[Developing an e-learning platform using sprin...,Developing an e-learning platform using spring...,,,,,,,...,unknown,,,,,,,,,
4,Software Engineer,"[Airbyte, Extract, Transform, Load (ETL), MySQ...","[-Deploy, improvise, monitor and maintain data...","-Deploy, improvise, monitor and maintain data ...","-Configure, create and maintain data pipelines...",- Google Cloud Management Platform Data servic...,-Develop and improve APIs,-Code review,,,...,unknown,,,,,,,,,
5,Software Engineer,"[Microsoft Word, SolidWorks, Web Design, Datab...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
6,Software Engineer,"[English, Engineering, Software Development, W...",[unknown],unknown,,,,,,,...,unknown,,,,,,,,,
7,Software Engineer,"[Python (Programming Language), Flask, MongoDB...",[Skills: Domain-Driven Design (DDD) · CI/CD · ...,Skills: Domain-Driven Design (DDD) · CI/CD · E...,,,,,,,...,المهارا的: docker · Spring Data · Spring Securi...,,,,,,,,,
8,Software Engineer,"[Machine Learning, Data Science, Python, Softw...",[unknown],unknown,,,,,,,...,Development of an E-commerce website<houchena>,,,,,,,,,
9,Software Engineer,"[Scrum, Databases, Jira, Linux, Data Science, ...",[• Analyze and elaborate requirements specific...,• Analyze and elaborate requirements specifica...,• Design and implementation of new functions.,• Perform unit tests.,• Support and functional support.,"Keywords: Python, JAVA, PL/SQL, SCRUM, Shell S...",,,...,unknown,,,,,,,,,


In [ ]:
#cluster_3achwaii

In [ ]:
# Filter data to include only data points belonging to cluster 2
cluster_data6 = datar2[datar2['cluster'] == 0].drop(columns=['cluster'])
cluster_data6

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# Define a function to calculate the Pearson correlation coefficient between two strings
def pearson_sim(s1, s2):
    s1 = s1.ljust(len(s2))
    s2 = s2.ljust(len(s1))
    s1 = [ord(c) for c in s1]
    s2 = [ord(c) for c in s2]
    return pearsonr(s1, s2)[0]

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data6['jobTitle'].tolist()
similarity_matrix = [[pearson_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]
print(similarity_matrix)
# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if ( (similarity_matrix[i][j] < 0.99 ) & (similarity_matrix[i][j] >= 0.6 ))]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data6.iloc[i],cluster_data6.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df6 = pd.concat(unique_combined_rows, axis=1).transpose()


In [59]:
concatenated_df = pd.concat([combined_dfBI,combined_dfCLOUD,combined_dfTWIN,combined_dfDATASCIENCE,combined_dfGL])
concatenated_df

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,Activite8,Activite9,Activite10
0,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,...,"Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
1,Internal Business Intelligence,"[Redmine, Microsoft Power BI, Talend Open Stud...",[Design and implementation of dashboards to an...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,...,unknown,,,,,,,,,
2,Business Intelligence Developer,"[Microsoft Power BI, Python (programming langu...",[Development of a BI solution to help detect f...,Development of a BI solution to help detect fr...,this solution based on 3 steps:,1/Data prepertion: data clearing with Python a...,2/Elaborate the modelling phase,3/Creating a predictive model,,,...,unknown,,,,,,,,,
3,Business Intelligence Internal,"[DevOps, Angular, Spring Boot, Microsoft Power...",[Skills: Business Intelligence (BI) · Web Deve...,Skills: Business Intelligence (BI) · Web Devel...,,,,,,,...,Data modelling and data warehousing.,"""Implement ETL processes and pipelines.",Data Reporting and visualization.,的 Implementation supervised/unsupervised algor...,,,,,,
4,Business Intelligence Developer,"[SQL Server Integration Services (SSIS), SQL S...",[unknown],unknown,,,,,,,...,"Skills: Extract, Transform, Load (ETL) · Micro...",,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Software Engineer,"[Next.js, React.js, Front-End Development, Web...",[unknown],unknown,,,,,,,...,المهاراю: Object-oriented programming (POO) · ...,,,,,,,,,
29,Software Developer,"[Python, Spring Boot, Programming, docker, Spr...",[المهارا的: docker · Spring Data · Spring Secur...,المهارا的: docker · Spring Data · Spring Securi...,,,,,,,...,• Analyze the functional and technical specifi...,• Manually test the user stories delivered by ...,• Execution of manual tests of APIs developed ...,• Collaborate with developers to ensure bindin...,• Write and keep up to date documentation.,,,,,
30,Software Developer,"[Oracle Database, Spring Boot, GitHub, Laravel...",[Development of an E-commerce website<houchena>],Development of an E-commerce website<houchena>,,,,,,,...,unknown,,,,,,,,,
31,Software Engineer,"[Teamwork, Team Leadership, Leadership, Proble...",[المهارا的: Spring Boot · keyclock · OpenSearch...,المهارا的: Spring Boot · keyclock · OpenSearch ...,,,,,,,...,"Design Dashboards, developer, test, implementa...",,,,,,,,,


In [60]:
concatenated_df.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/processed/lekhranchlh.csv")

### PRETRAITEMENT NLP

In [63]:
dataclean=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/processed/data_clean.csv")

In [65]:
dataclean.columns

Index(['Unnamed: 0', 'jobTitle', 'skills', 'Activity', 'Activite1',
       'Activite2', 'Activite3', 'Activite4', 'Activite5', 'Activite6',
       'Activite7', 'Activite8', 'Activite9', 'Activite10', 'jobTitle.1',
       'skills.1', 'Activity.1', 'Activite1.1', 'Activite2.1', 'Activite3.1',
       'Activite4.1', 'Activite5.1', 'Activite6.1', 'Activite7.1',
       'Activite8.1', 'Activite9.1', 'Activite10.1', 'word_tokenized1',
       'word_tokenized2', 'word_tokenized3', 'word_tokenized4',
       'word_tokenized5', 'word_tokenized6', 'word_tokenized7',
       'word_tokenized8', 'word_tokenized9', 'word_tokenized10',
       'word_tokenized11', 'word_tokenized12', 'word_tokenized13',
       'word_tokenized14', 'word_tokenized15', 'word_tokenized16',
       'word_tokenized17', 'word_tokenized18', 'word_tokenized19',
       'word_tokenized20', 'word_tokenized21', 'clean_stemmed1',
       'clean_stemmed2', 'clean_stemmed3', 'clean_stemmed4', 'clean_stemmed5',
       'clean_stemmed6', 'clean

In [66]:
dataclean.drop(["Unnamed: 0"],axis=1,inplace=True)

In [67]:
dataclean.head()

,jobTitle,skills,Activity,Activite1,Activite2,Activite3,Activite4,Activite5,Activite6,Activite7,...,clean_lemmed12,clean_lemmed13,clean_lemmed14,clean_lemmed15,clean_lemmed16,clean_lemmed17,clean_lemmed18,clean_lemmed19,clean_lemmed20,clean_lemmed21
0,Business Intelligence Developer,"['SQL Server Integration Services (SSIS)', 'SQ...",['unknown'],unknown,unknown,unknown,unknown,unknown,unknown,unknown,...,"['Skills', 'Extract', 'Transform', 'Load', 'ET...",[],[],[],[],[],[],[],[],[]
1,Internal Business Intelligence,"['Redmine', 'Microsoft Power BI', 'Talend Open...",['Design and implementation of dashboards to a...,Design and implementation of dashboards to ana...,"> Dashboard with plots, charts, filters about ...",> Follow-up of the status of a task(Ticket) an...,> Team performance monitoring and the advancem...,> Advancement of the sprint.,> Generation of business production plans.,> Identification of project monitoring indicat...,...,[],[],[],[],[],[],[],[],[],[]
2,Business Intelligence Developer,"['Microsoft Power BI', 'Python (programming la...",['Development of a BI solution to help detect ...,Development of a BI solution to help detect fr...,this solution based on 3 steps:,1/Data prepertion: data clearing with Python a...,2/Elaborate the modelling phase,3/Creating a predictive model,unknown,unknown,...,[],[],[],[],[],[],[],[],[],[]
3,Business Intelligence Internal,"['DevOps', 'Angular', 'Spring Boot', 'Microsof...",['Skills: Business Intelligence (BI) · Web Dev...,Skills: Business Intelligence (BI) · Web Devel...,unknown,unknown,unknown,unknown,unknown,unknown,...,"['Data', 'modelling', 'data', 'warehousing']","['Implement', 'ETL', 'process', 'pipeline']","['Data', 'Reporting', 'visualization']","['的', 'Implementation', 'supervised', 'unsuper...",[],[],[],[],[],[]
4,Business Intelligence Developer,"['SQL Server Integration Services (SSIS)', 'SQ...",['unknown'],unknown,unknown,unknown,unknown,unknown,unknown,unknown,...,"['Skills', 'Extract', 'Transform', 'Load', 'ET...",[],[],[],[],[],[],[],[],[]


### Levenshtein similarity

In [ ]:
import pandas as pd
from Levenshtein import distance

# Define a function to calculate the Levenshtein distance similarity between two strings
def levenshtein_sim(s1, s2):
    len1 = len(s1)
    len2 = len(s2)
    matrix = [[0] * (len2+1) for _ in range(len1+1)]
    for i in range(len1+1):
        for j in range(len2+1):
            if i == 0:
                matrix[i][j] = j
            elif j == 0:
                matrix[i][j] = i
            elif s1[i-1] == s2[j-1]:
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] = 1 + min(matrix[i-1][j], matrix[i][j-1], matrix[i-1][j-1])
    return 1 - (matrix[len1][len2] / max(len1, len2))

# Compute the similarity matrix between jobTitle columns
job_titles = cluster_data['jobTitle'].tolist()
similarity_matrix = [[levenshtein_sim(job_titles[i], job_titles[j]) for j in range(len(job_titles))] for i in range(len(job_titles))]

# Find the indices of rows that have a similarity greater than 0.8
indices = [(i, j) for i in range(len(similarity_matrix)) for j in range(i+1, len(similarity_matrix)) if similarity_matrix[i][j] > 0.8]

# Create a list of unique combined rows
combined_rows = []
unique_combined_rows=[] 
for i, j in indices:
    # Check if we have already combined these rows
    if i in combined_rows or j in combined_rows:
        continue
        
    # Combine the two rows into a new row
    combined_row = pd.concat([cluster_data.iloc[i],cluster_data.iloc[j]])
    
    # Add the index of the combined row to the list of combined rows
    combined_rows.extend([i, j])
    
    # Append the combined row to the list of unique combined rows
    unique_combined_rows.append(combined_row)
    
# Create a new DataFrame from the list of unique combined rows
combined_df3 = pd.concat(unique_combined_rows, axis=1).transpose()


In [ ]:
combined_df3

In [ ]:
combined_df3["Activity"]